# データサイエンス100本ノック（構造化データ加工編） - Python
# for Google Colab

## 【注意】オリジナル版との変更点
1. Google Colab ではDockerを使えないので、2020.06.18時点の 100knocks-preprocess/docker/work/data にあるCSVファイルをPostgreSQLから入手する代わりに使います。
2. オリジナルのCSVデータ, geocode.csvの'latitude'列名の初めにスペースが入っていたため、それを削除しました。

   オリジナル(100knocks-preprocess ver.1.0): ' latitude' --> 'latitude'
   

4. オリジナルの解答を元に、必要のないlibraryはimportせず、そして必要なlibraryをAzureNotebookでインストールするように最初のセルを改変してあります。
5. また、SQLではなく上記CSVからデータを読み込むように'はじめに'の最初のセルを改変してあります。

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートと~~データベース（PostgreSQL）~~ 100knocks-preprocess/docker/work/data にあるCSVファイルからのデータ読み込みを行います。geocode.csvに変更を加えたため、またgit cloneをするとColabの簡便さを損なうため、noguhiro2002のgithubレポジトリから直接読み込みます。
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能)
- オリジナルの解答例を元に、必要なライブラリーをpipでインストールします。
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
# pipでオリジナルの解答に必要なライブラリーをインストール
# !pip install --upgrade pip
# !pip install -U pandas numpy scikit-learn imbalanced-learn

# pipでオリジナルの解答に必要なライブラリーをインポート
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

# データを github/noguhiro2002/100knocks-preprocess/work/data フォルダよりDataframe形式でRead
df_customer = pd.read_csv('../../stats/customer.csv')
df_category = pd.read_csv('../../stats/category.csv')
df_product = pd.read_csv('../../stats/product.csv')
df_receipt = pd.read_csv('../../stats/receipt.csv')
df_store = pd.read_csv('../../stats/store.csv')
df_geocode = pd.read_csv('../../stats/geocode.csv')

# 演習問題

---
> P-001: レシート明細のデータフレーム（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [4]:
df_receipt.head(10)

sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
0   20181103   1541203200   S14006         112               1   
1   20181118   1542499200   S13008        1132               2   
2   20170712   1499817600   S14028        1102               1   
3   20190205   1549324800   S14042        1132               1   
4   20180821   1534809600   S14025        1102               2   
5   20190605   1559692800   S13003        1112               1   
6   20181205   1543968000   S14024        1102               2   
7   20190922   1569110400   S14040        1102               1   
8   20170504   1493856000   S13020        1112               2   
9   20191010   1570665600   S14027        1102               1   

      customer_id  product_cd  quantity  amount  
0  CS006214000001  P070305012         1     158  
1  CS008415000097  P070701017         1      81  
2  CS028414000014  P060101005         1     170  
3  ZZ000000000000  P050301001         1      25  
4  CS025415000050  P060102007         1      90  
5  CS003515000195  P050102002         1     138  
6  CS024514000042  P080101005         1      30  
7  CS040415000178  P070501004         1     128  
8  ZZ000000000000  P071302010         1     770  
9  CS027514000015  P071101003         1     680

> P-002: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。

In [2]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']].head(10)

sales_ymd     customer_id  product_cd  amount
0   20181103  CS006214000001  P070305012     158
1   20181118  CS008415000097  P070701017      81
2   20170712  CS028414000014  P060101005     170
3   20190205  ZZ000000000000  P050301001      25
4   20180821  CS025415000050  P060102007      90
5   20190605  CS003515000195  P050102002     138
6   20181205  CS024514000042  P080101005      30
7   20190922  CS040415000178  P070501004     128
8   20170504  ZZ000000000000  P071302010     770
9   20191010  CS027514000015  P071101003     680

---
> P-003: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。ただし、sales_ymdはsales_dateに項目名を変更しながら抽出すること。

In [6]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']].rename(columns={'sales_ymd': 'sales_date'}).head(10)

sales_date     customer_id  product_cd  amount
0    20181103  CS006214000001  P070305012     158
1    20181118  CS008415000097  P070701017      81
2    20170712  CS028414000014  P060101005     170
3    20190205  ZZ000000000000  P050301001      25
4    20180821  CS025415000050  P060102007      90
5    20190605  CS003515000195  P050102002     138
6    20181205  CS024514000042  P080101005      30
7    20190922  CS040415000178  P070501004     128
8    20170504  ZZ000000000000  P071302010     770
9    20191010  CS027514000015  P071101003     680

---
> P-004: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [ ]:
# queryは、pandas では、つかわない。データの集まりを作って、操作する
# バグをださないよう、オリジナルをディープコピーして、かたまりわけてやるとわかりやすい

In [ ]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']].query('customer_id == "CS018205000001"')

sales_ymd     customer_id  product_cd  amount
36      20180911  CS018205000001  P071401012    2200
9843    20180414  CS018205000001  P060104007     600
21110   20170614  CS018205000001  P050206001     990
27673   20170614  CS018205000001  P060702015     108
27840   20190216  CS018205000001  P071005024     102
28757   20180414  CS018205000001  P071101002     278
39256   20190226  CS018205000001  P070902035     168
58121   20190924  CS018205000001  P060805001     495
68117   20190226  CS018205000001  P071401020    2200
72254   20180911  CS018205000001  P071401005    1100
88508   20190216  CS018205000001  P040101002     218
91525   20190924  CS018205000001  P091503001     280

In [ ]:
# df_receipt[df_receipt["customer_id"] == "CS018205000001"][["sales_ymd", "customer_id", "product_cd", "amount"]]
df_receipt[["sales_ymd", "customer_id", "product_cd", "amount"]][df_receipt["customer_id"] == "CS018205000001"]

sales_ymd     customer_id  product_cd  amount
36      20180911  CS018205000001  P071401012    2200
9843    20180414  CS018205000001  P060104007     600
21110   20170614  CS018205000001  P050206001     990
27673   20170614  CS018205000001  P060702015     108
27840   20190216  CS018205000001  P071005024     102
28757   20180414  CS018205000001  P071101002     278
39256   20190226  CS018205000001  P070902035     168
58121   20190924  CS018205000001  P060805001     495
68117   20190226  CS018205000001  P071401020    2200
72254   20180911  CS018205000001  P071401005    1100
88508   20190216  CS018205000001  P040101002     218
91525   20190924  CS018205000001  P091503001     280

---
> P-005: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [ ]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']].query('customer_id == "CS018205000001" & amount >= 1000')

sales_ymd     customer_id  product_cd  amount
36      20180911  CS018205000001  P071401012    2200
68117   20190226  CS018205000001  P071401020    2200
72254   20180911  CS018205000001  P071401005    1100

In [ ]:
# df_receipt[["sales_ymd", "customer_id", "product_cd", "amount"]][
#                                                                  (df_receipt["customer_id"] == "CS018205000001") & \
#                                                                  (df_receipt["amount"] >= 1000)]
df_receipt[(df_receipt["customer_id"] == "CS018205000001") &  (df_receipt["amount"] >= 1000)][["sales_ymd", "customer_id", "product_cd", "amount"]]

sales_ymd     customer_id  product_cd  amount
36      20180911  CS018205000001  P071401012    2200
68117   20190226  CS018205000001  P071401020    2200
72254   20180911  CS018205000001  P071401005    1100

In [ ]:
# オリジナルをコピー
df_05 =df_receipt.copy()
# 表示したい項目を絞る
df_05 = df_05[["sales_ymd", "customer_id", "product_cd", "amount"]]
# 条件で絞り込む () の中は条件式
df_05[(df_05["customer_id"]=="CS018205000001") & (df_05["amount"]>=1000)]

sales_ymd     customer_id  product_cd  amount
36      20180911  CS018205000001  P071401012    2200
68117   20190226  CS018205000001  P071401020    2200
72254   20180911  CS018205000001  P071401005    1100

---
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [ ]:
df_receipt[['sales_ymd','customer_id','product_cd','quantity','amount']].query('customer_id == "CS018205000001" & (amount >= 1000 | quantity >= 5)')

sales_ymd     customer_id  product_cd  quantity  amount
36      20180911  CS018205000001  P071401012         1    2200
9843    20180414  CS018205000001  P060104007         6     600
21110   20170614  CS018205000001  P050206001         5     990
68117   20190226  CS018205000001  P071401020         1    2200
72254   20180911  CS018205000001  P071401005         1    1100

In [ ]:
df_receipt[(df_receipt["customer_id"] == "CS018205000001") & ((df_receipt["amount"] >= 1000) | (df_receipt["quantity"] >= 5))]\
[['sales_ymd','customer_id','product_cd','quantity','amount']]

sales_ymd     customer_id  product_cd  quantity  amount
36      20180911  CS018205000001  P071401012         1    2200
9843    20180414  CS018205000001  P060104007         6     600
21110   20170614  CS018205000001  P050206001         5     990
68117   20190226  CS018205000001  P071401020         1    2200
72254   20180911  CS018205000001  P071401005         1    1100

In [ ]:
# オリジナルをコピー
df_06 =df_receipt.copy()
# 表示したい項目を絞る
df_06 = df_06[["sales_ymd", "customer_id", "product_cd","quantity", "amount"]]
# 条件で絞り込む () の中は条件式
df_06[(df_06["customer_id"]=="CS018205000001") & ((df_06["amount"]>=1000) | (df_06["quantity"] >=5))]

sales_ymd     customer_id  product_cd  quantity  amount
36      20180911  CS018205000001  P071401012         1    2200
9843    20180414  CS018205000001  P060104007         6     600
21110   20170614  CS018205000001  P050206001         5     990
68117   20190226  CS018205000001  P071401020         1    2200
72254   20180911  CS018205000001  P071401005         1    1100

---
> P-007: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [ ]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']].query('customer_id == "CS018205000001" & 1000<= amount <= 2000')

sales_ymd     customer_id  product_cd  amount
72254   20180911  CS018205000001  P071401005    1100

In [ ]:
df_receipt[(df_receipt["customer_id"] == "CS018205000001") & (1000 <= df_receipt["amount"]) & (df_receipt["amount"]<= 2000)]\
[['sales_ymd','customer_id','product_cd','amount']]

sales_ymd     customer_id  product_cd  amount
72254   20180911  CS018205000001  P071401005    1100

In [ ]:
# オリジナルをコピー
df_07 =df_receipt.copy()
# 表示したい項目を絞る
df_07 = df_07[["sales_ymd", "customer_id", "product_cd","amount"]]
# 条件で絞り込む () の中は条件式
df_07[(df_07["customer_id"]=="CS018205000001") 
& (df_07["amount"]>=1000) & (df_07["amount"] <=2000)]

sales_ymd     customer_id  product_cd  amount
72254   20180911  CS018205000001  P071401005    1100

---
> P-008: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [ ]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']].query('customer_id == "CS018205000001" & product_cd != "P071401019"')

sales_ymd     customer_id  product_cd  amount
36      20180911  CS018205000001  P071401012    2200
9843    20180414  CS018205000001  P060104007     600
21110   20170614  CS018205000001  P050206001     990
27673   20170614  CS018205000001  P060702015     108
27840   20190216  CS018205000001  P071005024     102
28757   20180414  CS018205000001  P071101002     278
39256   20190226  CS018205000001  P070902035     168
58121   20190924  CS018205000001  P060805001     495
68117   20190226  CS018205000001  P071401020    2200
72254   20180911  CS018205000001  P071401005    1100
88508   20190216  CS018205000001  P040101002     218
91525   20190924  CS018205000001  P091503001     280

In [ ]:
# オリジナルをコピー
df_08 =df_receipt.copy()
# 表示したい項目を絞る
df_08 = df_08[["sales_ymd", "customer_id", "product_cd","amount"]]
# 条件で絞り込む () の中は条件式
df_08[(df_08["customer_id"]=="CS018205000001") 
    & (df_08["product_cd"] != "P071401019")]

sales_ymd     customer_id  product_cd  amount
36      20180911  CS018205000001  P071401012    2200
9843    20180414  CS018205000001  P060104007     600
21110   20170614  CS018205000001  P050206001     990
27673   20170614  CS018205000001  P060702015     108
27840   20190216  CS018205000001  P071005024     102
28757   20180414  CS018205000001  P071101002     278
39256   20190226  CS018205000001  P070902035     168
58121   20190924  CS018205000001  P060805001     495
68117   20190226  CS018205000001  P071401020    2200
72254   20180911  CS018205000001  P071401005    1100
88508   20190216  CS018205000001  P040101002     218
91525   20190924  CS018205000001  P091503001     280

---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。

`df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [ ]:
df_store.query('not(prefecture_cd == "13" | floor_area > 900)')

store_cd store_name  prefecture_cd  ... longitude  latitude floor_area
18   S14046       北山田店             14  ...  139.5916  35.56189      831.0
20   S14011      日吉本町店             14  ...  139.6316  35.54655      890.0
38   S12013       習志野店             12  ...  140.0220  35.66122      808.0

[3 rows x 10 columns]

In [ ]:
df_store.query('prefecture_cd != "13" & floor_area <= 900')

store_cd store_name  prefecture_cd  ... longitude  latitude floor_area
18   S14046       北山田店             14  ...  139.5916  35.56189      831.0
20   S14011      日吉本町店             14  ...  139.6316  35.54655      890.0
38   S12013       習志野店             12  ...  140.0220  35.66122      808.0

[3 rows x 10 columns]

In [ ]:
df_09 = df_store.copy()
df_09[(df_09["prefecture_cd"] != 13)&(df_09["floor_area"] <= 900)]


store_cd store_name  prefecture_cd  ... longitude  latitude floor_area
18   S14046       北山田店             14  ...  139.5916  35.56189      831.0
20   S14011      日吉本町店             14  ...  139.6316  35.54655      890.0
38   S12013       習志野店             12  ...  140.0220  35.66122      808.0

[3 rows x 10 columns]

---
> P-010: 店舗データフレーム（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件だけ表示せよ。

In [ ]:
df_store[df_store['store_cd'].str.startswith('S14')].head(10)

store_cd store_name  prefecture_cd  ... longitude  latitude floor_area
2    S14010        菊名店             14  ...  139.6326  35.50049     1732.0
3    S14033       阿久和店             14  ...  139.4961  35.45918     1495.0
4    S14036     相模原中央店             14  ...  139.3716  35.57327     1679.0
7    S14040       長津田店             14  ...  139.4994  35.52398     1548.0
9    S14050      阿久和西店             14  ...  139.4961  35.45918     1830.0
12   S14028       二ツ橋店             14  ...  139.4963  35.46304     1574.0
16   S14012      本牧和田店             14  ...  139.6582  35.42156     1341.0
18   S14046       北山田店             14  ...  139.5916  35.56189      831.0
19   S14022        逗子店             14  ...  139.5789  35.29642     1838.0
20   S14011      日吉本町店             14  ...  139.6316  35.54655      890.0

[10 rows x 10 columns]

In [ ]:
df_store.query('store_cd.str.startswith("S14")', engine='python').head(10)

store_cd store_name  prefecture_cd  ... longitude  latitude floor_area
2    S14010        菊名店             14  ...  139.6326  35.50049     1732.0
3    S14033       阿久和店             14  ...  139.4961  35.45918     1495.0
4    S14036     相模原中央店             14  ...  139.3716  35.57327     1679.0
7    S14040       長津田店             14  ...  139.4994  35.52398     1548.0
9    S14050      阿久和西店             14  ...  139.4961  35.45918     1830.0
12   S14028       二ツ橋店             14  ...  139.4963  35.46304     1574.0
16   S14012      本牧和田店             14  ...  139.6582  35.42156     1341.0
18   S14046       北山田店             14  ...  139.5916  35.56189      831.0
19   S14022        逗子店             14  ...  139.5789  35.29642     1838.0
20   S14011      日吉本町店             14  ...  139.6316  35.54655      890.0

[10 rows x 10 columns]

---
> P-011: 顧客データフレーム（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件だけ表示せよ。

In [ ]:
df_customer[df_customer['customer_id'].str.endswith('1')].head(10)

customer_id customer_name  ...  application_date     status_cd
1    CS037613000071         六角 雅彦  ...          20150414  0-00000000-0
3    CS028811000001        堀井 かおり  ...          20160115  0-00000000-0
14   CS040412000191         川井 郁恵  ...          20151101  1-20091025-4
31   CS028314000011        小菅 あおい  ...          20151123  1-20080426-5
56   CS039212000051        藤島 恵梨香  ...          20171121  1-20100215-4
59   CS015412000111         松居 奈月  ...          20150629  0-00000000-0
63   CS004702000041          野島 洋  ...          20170218  0-00000000-0
74   CS041515000001         栗田 千夏  ...          20160422  E-20100803-F
85   CS029313000221        北条 ひかり  ...          20180810  0-00000000-0
102  CS034312000071         望月 奈央  ...          20160106  0-00000000-0

[10 rows x 11 columns]

---
> P-012: 店舗データフレーム（df_store）から横浜市の店舗だけ全項目表示せよ。

In [ ]:
df_store[df_store['address'].str.contains('横浜市')]

store_cd store_name  prefecture_cd  ... longitude  latitude floor_area
2    S14010        菊名店             14  ...  139.6326  35.50049     1732.0
3    S14033       阿久和店             14  ...  139.4961  35.45918     1495.0
7    S14040       長津田店             14  ...  139.4994  35.52398     1548.0
9    S14050      阿久和西店             14  ...  139.4961  35.45918     1830.0
12   S14028       二ツ橋店             14  ...  139.4963  35.46304     1574.0
16   S14012      本牧和田店             14  ...  139.6582  35.42156     1341.0
18   S14046       北山田店             14  ...  139.5916  35.56189      831.0
20   S14011      日吉本町店             14  ...  139.6316  35.54655      890.0
26   S14048      中川中央店             14  ...  139.5758  35.54912     1657.0
40   S14042       新山下店             14  ...  139.6593  35.43894     1044.0
52   S14006       葛が谷店             14  ...  139.5633  35.53573     1886.0

[11 rows x 10 columns]

---
> P-013: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件だけ表示せよ。

In [ ]:
df_customer[df_customer['status_cd'].str.match(r"^[A-F]")]

customer_id customer_name  ...  application_date     status_cd
2   CS031415000172       宇多田 貴美子  ...          20150529  D-20100325-C
6   CS015414000103         奥野 陽子  ...          20150722  B-20100609-B
12  CS011215000048         芦田 沙耶  ...          20150228  C-20100421-9
15  CS029415000023         梅田 里穂  ...          20150610  D-20100918-E
21  CS035415000029         寺沢 真希  ...          20141220  F-20101029-F
32  CS031415000106        宇野 由美子  ...          20150201  F-20100511-E
33  CS029215000025         石倉 美帆  ...          20150708  B-20100820-C
40  CS033605000005         猪股 雄太  ...          20150425  F-20100917-E
44  CS033415000229        板垣 菜々美  ...          20150712  F-20100326-E
53  CS008415000145         黒谷 麻緒  ...          20150829  F-20100622-F

[10 rows x 11 columns]

---
> P-014: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [ ]:
df_customer[df_customer['status_cd'].str.contains(r"[1-9]$")].head(10)

customer_id customer_name  ...  application_date     status_cd
4   CS001215000145         田崎 美紀  ...          20170605  6-20090929-2
9   CS033513000180          安斎 遥  ...          20150728  6-20080506-5
12  CS011215000048         芦田 沙耶  ...          20150228  C-20100421-9
14  CS040412000191         川井 郁恵  ...          20151101  1-20091025-4
16  CS009315000023         皆川 文世  ...          20150319  5-20080322-1
22  CS015315000033        福士 璃奈子  ...          20141024  4-20080219-3
23  CS023513000066         神戸 そら  ...          20150915  5-20100524-9
24  CS035513000134         市川 美帆  ...          20150227  8-20100711-9
27  CS001515000263         高松 夏空  ...          20160812  1-20100804-1
28  CS040314000027       鶴田 きみまろ  ...          20150122  2-20080426-4

[10 rows x 11 columns]

In [ ]:
df_customer[df_customer['status_cd'].str.match(r".*[1-9]$")].head(10)

customer_id customer_name  ...  application_date     status_cd
4   CS001215000145         田崎 美紀  ...          20170605  6-20090929-2
9   CS033513000180          安斎 遥  ...          20150728  6-20080506-5
12  CS011215000048         芦田 沙耶  ...          20150228  C-20100421-9
14  CS040412000191         川井 郁恵  ...          20151101  1-20091025-4
16  CS009315000023         皆川 文世  ...          20150319  5-20080322-1
22  CS015315000033        福士 璃奈子  ...          20141024  4-20080219-3
23  CS023513000066         神戸 そら  ...          20150915  5-20100524-9
24  CS035513000134         市川 美帆  ...          20150227  8-20100711-9
27  CS001515000263         高松 夏空  ...          20160812  1-20100804-1
28  CS040314000027       鶴田 きみまろ  ...          20150122  2-20080426-4

[10 rows x 11 columns]

---
> P-015: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [ ]:
df_customer[df_customer['status_cd'].str.match(r"^[A-F].*[1-9]$")].head(10)

customer_id customer_name  ...  application_date     status_cd
12   CS011215000048         芦田 沙耶  ...          20150228  C-20100421-9
68   CS022513000105        島村 貴美子  ...          20150320  A-20091115-7
71   CS001515000096         水野 陽子  ...          20150614  A-20100724-7
122  CS013615000053         西脇 季衣  ...          20150128  B-20100329-6
144  CS020412000161          小宮 薫  ...          20150822  B-20081021-3
178  CS001215000097        竹中 あさみ  ...          20170315  A-20100211-2
252  CS035212000007        内村 恵梨香  ...          20151013  B-20101018-6
259  CS002515000386         野田 コウ  ...          20160410  C-20100127-8
293  CS001615000372        稲垣 寿々花  ...          20170403  A-20100104-1
297  CS032512000121         松井 知世  ...          20150727  A-20100103-5

[10 rows x 11 columns]

---
> P-016: 店舗データフレーム（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [ ]:
df_store[df_store['tel_no'].str.match(r'\d{3}-\d{3}-\d{4}')]

store_cd store_name  prefecture_cd  ... longitude  latitude floor_area
0    S12014       千草台店             12  ...  140.1180  35.63559     1698.0
1    S13002       国分寺店             13  ...  139.4802  35.70566     1735.0
2    S14010        菊名店             14  ...  139.6326  35.50049     1732.0
3    S14033       阿久和店             14  ...  139.4961  35.45918     1495.0
4    S14036     相模原中央店             14  ...  139.3716  35.57327     1679.0
7    S14040       長津田店             14  ...  139.4994  35.52398     1548.0
9    S14050      阿久和西店             14  ...  139.4961  35.45918     1830.0
11   S13052        森野店             13  ...  139.4383  35.55293     1087.0
12   S14028       二ツ橋店             14  ...  139.4963  35.46304     1574.0
16   S14012      本牧和田店             14  ...  139.6582  35.42156     1341.0
18   S14046       北山田店             14  ...  139.5916  35.56189      831.0
19   S14022        逗子店             14  ...  139.5789  35.29642     1838.0
20   S14011      日吉本町店             14  ...  139.6316  35.54655      890.0
21   S13016       小金井店             13  ...  139.5094  35.70018     1399.0
22   S14034      川崎野川店             14  ...  139.5998  35.57693     1318.0
26   S14048      中川中央店             14  ...  139.5758  35.54912     1657.0
27   S12007        佐倉店             12  ...  140.1452  35.71872     1895.0
28   S14026     辻堂西海岸店             14  ...  139.4466  35.32464     1732.0
29   S13041       八王子店             13  ...  139.4235  35.63787      810.0
31   S14049      川崎大師店             14  ...  139.7327  35.53759      962.0
32   S14023        川崎店             14  ...  139.7028  35.53599     1804.0
33   S13018        清瀬店             13  ...  139.5178  35.76885     1220.0
35   S14027       南藤沢店             14  ...  139.4896  35.33762     1521.0
36   S14021       伊勢原店             14  ...  139.3129  35.40169      962.0
37   S14047       相模原店             14  ...  139.3748  35.55959     1047.0
38   S12013       習志野店             12  ...  140.0220  35.66122      808.0
40   S14042       新山下店             14  ...  139.6593  35.43894     1044.0
42   S12030        八幡店             12  ...  139.9240  35.72318     1162.0
44   S14025        大和店             14  ...  139.4680  35.43414     1011.0
45   S14045        厚木店             14  ...  139.3651  35.44182      980.0
47   S12029        東野店             12  ...  139.8968  35.65086     1101.0
49   S12053        高洲店             12  ...  139.9176  35.63755     1555.0
51   S14024        三田店             14  ...  139.5424  35.60770      972.0
52   S14006       葛が谷店             14  ...  139.5633  35.53573     1886.0

[34 rows x 10 columns]

---
> P-17: 顧客データフレーム（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭10件を全項目表示せよ。

In [ ]:
df_customer.sort_values('birth_day').head(10)

customer_id customer_name  ...  application_date     status_cd
18817  CS003813000014        村山 菜々美  ...          20160214  0-00000000-0
12328  CS026813000004         吉村 朝陽  ...          20150723  0-00000000-0
15682  CS018811000003         熊沢 美里  ...          20150403  0-00000000-0
15302  CS027803000004         内村 拓郎  ...          20151227  0-00000000-0
1681   CS013801000003         天野 拓郎  ...          20160120  0-00000000-0
7511   CS001814000022         鶴田 里穂  ...          20161012  A-20090415-7
2378   CS016815000002         山元 美紀  ...          20150629  C-20090923-C
4680   CS009815000003         中田 里穂  ...          20150421  D-20091021-E
16070  CS005813000015        金谷 恵梨香  ...          20150506  0-00000000-0
6305   CS012813000013         宇野 南朋  ...          20150712  0-00000000-0

[10 rows x 11 columns]

---
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭10件を全項目表示せよ。

In [ ]:
df_customer.sort_values('birth_day',ascending=False).head(10)

customer_id customer_name  ...  application_date     status_cd
15639  CS035114000004         大村 美里  ...          20150619  6-20091205-6
7468   CS022103000002        福山 はじめ  ...          20160909  0-00000000-0
10745  CS002113000009        柴田 真悠子  ...          20160304  0-00000000-0
19811  CS004115000014         松井 京子  ...          20161120  1-20081231-1
7039   CS002114000010          山内 遥  ...          20160920  6-20100510-1
3670   CS025115000002         小柳 夏希  ...          20160116  D-20100913-D
12493  CS002113000025        広末 まなみ  ...          20171030  0-00000000-0
15977  CS033112000003         長野 美紀  ...          20150606  0-00000000-0
5716   CS007115000006          福岡 瞬  ...          20151118  F-20101016-F
15097  CS014113000008         矢口 莉緒  ...          20150622  3-20091108-6

[10 rows x 11 columns]

---
> P-19: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [ ]:
# method='min'とすると最小値が順位となる。1位、2位タイ、2位タイ、4位
df_receipt['rank_amount']=df_receipt['amount'].rank(method='min', ascending=False)

In [ ]:
df_receipt['rank_amount'].sort_values().head(10)

1202     1.0
62317    2.0
54095    3.0
4632     4.0
72747    4.0
10320    4.0
97294    7.0
28304    7.0
92246    9.0
68553    9.0
Name: rank_amount, dtype: float64

In [ ]:
df_receipt.sort_values('rank_amount')[['customer_id','amount','rank_amount']].head(10)


customer_id  amount  rank_amount
1202   CS011415000006   10925          1.0
62317  ZZ000000000000    6800          2.0
54095  CS028605000002    5780          3.0
4632   CS015515000034    5480          4.0
72747  ZZ000000000000    5480          4.0
10320  ZZ000000000000    5480          4.0
97294  CS021515000089    5440          7.0
28304  ZZ000000000000    5440          7.0
92246  CS009415000038    5280          9.0
68553  CS040415000200    5280          9.0

---
> P-020: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [ ]:
#first：同一値は登場順に順位付け。数値のみに有効。
df_receipt['rank_amount2']=df_receipt['amount'].rank(method='first', ascending=False)

In [ ]:
df_receipt.sort_values('rank_amount2')[['customer_id','amount','rank_amount2']].head(10)


customer_id  amount  rank_amount2
1202   CS011415000006   10925           1.0
62317  ZZ000000000000    6800           2.0
54095  CS028605000002    5780           3.0
4632   CS015515000034    5480           4.0
10320  ZZ000000000000    5480           5.0
72747  ZZ000000000000    5480           6.0
28304  ZZ000000000000    5440           7.0
97294  CS021515000089    5440           8.0
596    CS015515000083    5280           9.0
11275  CS017414000114    5280          10.0

---
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [ ]:
df_receipt.count()

sales_ymd         104681
sales_epoch       104681
store_cd          104681
receipt_no        104681
receipt_sub_no    104681
customer_id       104681
product_cd        104681
quantity          104681
amount            104681
rank_amount       104681
rank_amount2      104681
dtype: int64

In [ ]:
df_receipt["receipt_no"].count()

104681

In [ ]:
len(df_receipt)


104681

In [ ]:
df_receipt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104681 entries, 0 to 104680
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   sales_ymd       104681 non-null  int64  
 1   sales_epoch     104681 non-null  int64  
 2   store_cd        104681 non-null  object 
 3   receipt_no      104681 non-null  int64  
 4   receipt_sub_no  104681 non-null  int64  
 5   customer_id     104681 non-null  object 
 6   product_cd      104681 non-null  object 
 7   quantity        104681 non-null  int64  
 8   amount          104681 non-null  int64  
 9   rank_amount     104681 non-null  float64
dtypes: float64(1), int64(6), object(3)
memory usage: 8.0+ MB


---
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [ ]:
df_022 = df_receipt.copy()
len(df_022[["customer_id"]].groupby("customer_id"))

#groupbyで新しい塊のオブジェクトを作っただけで、なにも返さない。どんな形で返したいか、指示をだす

#unique重複ないデータを取りたい時→lenで囲って配列の数だす
#len(df_receipt['customer_id'].unique())

8307

In [ ]:
#nunique重複ないデータのカウントを取りたいとき
df_receipt['customer_id'].nunique()

8307

---
> P-023: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [ ]:
#list(df_receipt.groupby("store_cd"))
#list(df_receipt.groupby("store_cd"))[0]
#list(df_receipt.groupby("store_cd"))[0][0]
#list(df_receipt.groupby("store_cd"))[0][1]
#list(df_receipt.groupby("store_cd"))[0][1][["amount""quantity"]].sum()

In [40]:
df_023 = df_receipt.copy()
df_023.groupby("store_cd")[["amount","quantity"]].sum()




amount  quantity
store_cd                  
S12007    638761      2099
S12013    787513      2425
S12014    725167      2358
S12029    794741      2555
S12030    684402      2403
S13001    811936      2347
S13002    727821      2340
S13003    764294      2197
S13004    779373      2390
S13005    629876      2004
S13008    809288      2491
S13009    808870      2486
S13015    780873      2248
S13016    793773      2432
S13017    748221      2376
S13018    790535      2562
S13019    827833      2541
S13020    796383      2383
S13031    705968      2336
S13032    790501      2491
S13035    715869      2219
S13037    693087      2344
S13038    708884      2337
S13039    611888      1981
S13041    728266      2233
S13043    587895      1881
S13044    520764      1729
S13051    107452       354
S13052    100314       250
S14006    712839      2284
S14010    790361      2290
S14011    805724      2434
S14012    720600      2412
S14021    699511      2231
S14022    651328      2047
S14023    727630      2258
S14024    736323      2417
S14025    755581      2394
S14026    824537      2503
S14027    714550      2303
S14028    786145      2458
S14033    725318      2282
S14034    653681      2024
S14036    203694       635
S14040    701858      2233
S14042    534689      1935
S14045    458484      1398
S14046    412646      1354
S14047    338329      1041
S14048    234276       769
S14049    230808       788
S14050    167090       580

---
> P-024: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）を求め、10件表示せよ。

In [ ]:
df_024 = df_receipt.copy()
#日付型に変更
df_024["sales_ymd"] = pd.to_datetime(df_024["sales_ymd"],format="%Y%m%d") 
#日付型に変更されたか確認
df_024["sales_ymd"]
#顧客ID（customer_id）ごとに最も新しい売上日の頭10件表示
df_024.groupby("customer_id")["sales_ymd"].max().head(10)

customer_id
CS001113000004   2019-03-08
CS001114000005   2019-07-31
CS001115000010   2019-04-05
CS001205000004   2019-06-25
CS001205000006   2019-02-24
CS001211000025   2019-03-22
CS001212000027   2017-01-27
CS001212000031   2018-09-06
CS001212000046   2017-08-11
CS001212000070   2019-10-18
Name: sales_ymd, dtype: datetime64[ns]

---
> P-025: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上日（sales_ymd）を求め、10件表示せよ。

In [ ]:
df_025 = df_receipt.copy()
df_025["sales_ymd"] = pd.to_datetime(df_025["sales_ymd"],format="%Y%m%d") 
df_025["sales_ymd"]
df_025.groupby("customer_id")["sales_ymd"].min().head(10)


customer_id
CS001113000004   2019-03-08
CS001114000005   2018-05-03
CS001115000010   2017-12-28
CS001205000004   2017-09-14
CS001205000006   2018-02-07
CS001211000025   2019-03-22
CS001212000027   2017-01-27
CS001212000031   2018-09-06
CS001212000046   2017-08-11
CS001212000070   2019-10-18
Name: sales_ymd, dtype: datetime64[ns]

---
> P-026: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）と古い売上日を求め、両者が異なるデータを10件表示せよ。

In [ ]:
df_026 = df_receipt.copy()
df_026["sales_ymd"] = pd.to_datetime(df_026["sales_ymd"],format="%Y%m%d") 
#キー(customer_id)とminの塊つくる
s_026_ymd_min = df_026.groupby("customer_id")["sales_ymd"].min()
s_026_ymd_min.name="sales_ymd_min"

#キー(customer_id)とmaxの塊つくる
s_026_ymd_max = df_026.groupby("customer_id")["sales_ymd"].max()
s_026_ymd_max.name="sales_ymd_max"
#concat 塊をくっつけるから、[]でくくる。書いてある順に塊をくっつける。axis=1がないと縦にくっつける。axis=1 で横に１つくっつける
#キーが同じもの同士くっつけるときに列名を変える→キーが全部同じだから、concat使える。違う場合は、merge（ゆかりおさんの参照）
df_026_con = pd.concat([s_026_ymd_min,s_026_ymd_max],axis=1)

# キーとTrue or Falesか結果をだす
# df_026_con["sales_ymd_min"] != df_026_con["sales_ymd_max"]
#↓
#Trueになるものだけだす
df_026_con[df_026_con["sales_ymd_min"] != df_026_con["sales_ymd_max"]].head(10)


sales_ymd_min sales_ymd_max
customer_id                               
CS001114000005    2018-05-03    2019-07-31
CS001115000010    2017-12-28    2019-04-05
CS001205000004    2017-09-14    2019-06-25
CS001205000006    2018-02-07    2019-02-24
CS001214000009    2017-03-06    2019-09-02
CS001214000017    2018-08-28    2019-10-06
CS001214000048    2017-11-09    2019-09-29
CS001214000052    2018-02-08    2019-06-17
CS001215000005    2017-02-06    2018-10-21
CS001215000040    2017-02-14    2017-10-22

---
> P-027: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [195]:
df_027 = df_receipt.copy()
df_027.groupby("store_cd")[["amount"]].mean().sort_values("amount",ascending=False).head(5)


amount
store_cd            
S13052    402.867470
S13015    351.111960
S13003    350.915519
S14010    348.791262
S13001    348.470386

# P-28

---
> P-028: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [197]:
df_028 = df_receipt.copy()
df_028.groupby("store_cd")[["amount"]].median().sort_values("amount",ascending=False).head()

amount
store_cd        
S13052       190
S14010       188
S14050       185
S14040       180
S13003       180

# P-29

---
> P-029: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求めよ。

In [200]:
df29 = df_receipt.copy()
# groupby の中身をみたいときは、リストでみれる。1個目：グループ化したときのキー、2個目：グループ化の対象の塊
# list(df29.groupby(["store_cd", "product_cd"]))
df29.groupby("store_cd")[["product_cd"]].apply(lambda x: x.mode())
#groupbyが提供しているメソッドに限りがある。applyで1つ1つのセルごとに関数を適用している。
# ラムダ式
#https://www.atmarkit.co.jp/ait/articles/1908/26/news124.html


product_cd
store_cd              
S12007   0  P060303001
S12013   0  P060303001
S12014   0  P060303001
S12029   0  P060303001
S12030   0  P060303001
S13001   0  P060303001
S13002   0  P060303001
S13003   0  P071401001
S13004   0  P060303001
S13005   0  P040503001
S13008   0  P060303001
S13009   0  P060303001
S13015   0  P071401001
S13016   0  P071102001
S13017   0  P060101002
S13018   0  P071401001
S13019   0  P071401001
S13020   0  P071401001
S13031   0  P060303001
S13032   0  P060303001
S13035   0  P040503001
S13037   0  P060303001
S13038   0  P060303001
S13039   0  P071401001
S13041   0  P071401001
S13043   0  P060303001
S13044   0  P060303001
S13051   0  P050102001
         1  P071003001
         2  P080804001
S13052   0  P050101001
S14006   0  P060303001
S14010   0  P060303001
S14011   0  P060101001
S14012   0  P060303001
S14021   0  P060101001
S14022   0  P060303001
S14023   0  P071401001
S14024   0  P060303001
S14025   0  P060303001
S14026   0  P071401001
S14027   0  P060303001
S14028   0  P060303001
S14033   0  P071401001
S14034   0  P060303001
S14036   0  P040503001
         1  P060101001
S14040   0  P060303001
S14042   0  P050101001
S14045   0  P060303001
S14046   0  P060303001
S14047   0  P060303001
S14048   0  P050101001
S14049   0  P060303001
S14050   0  P060303001

In [ ]:
w = pd.DataFrame(df29.groupby(["store_cd", "product_cd"]).size(),columns=['cnt'])
w



cnt
store_cd product_cd     
S12007   P040101001   11
         P040101002    5
         P040101003    3
         P040101005    3
         P040101008    1
...                  ...
S14050   P091501002    1
         P091501036    1
         P091503001    7
         P091503002    1
         P091503004    1

[54194 rows x 1 columns]

# P-30

---
> P-030: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本分散を計算し、降順でTOP5を表示せよ。

In [ ]:
df_30 = df_receipt.copy()
df_30.groupby("store_cd")["amount"].var(ddof=0).sort_values(ascending=False).head()

store_cd
S13052    440088.701311
S14011    306314.558164
S14034    296920.081011
S13001    295431.993329
S13015    295294.361116
Name: amount, dtype: float64

# P-31

---
> P-031: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本標準偏差を計算し、降順でTOP5を表示せよ。

In [ ]:
df_31 = df_receipt.copy()
df_31.groupby("store_cd")["amount"].std(ddof=0).sort_values(ascending=False).head()

store_cd
S13052    663.391816
S14011    553.456916
S14034    544.903736
S13001    543.536561
S13015    543.409938
Name: amount, dtype: float64

# P-32

---
> P-032: レシート明細データフレーム（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [ ]:
df_32 = df_receipt.copy()
df_31["amount"].quantile([0, 0.25, 0.5, 0.75, 1.0])

0.00       10.0
0.25      102.0
0.50      170.0
0.75      288.0
1.00    10925.0
Name: amount, dtype: float64

# P-33

---
> P-033: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [2]:
df_33 = df_receipt.copy()
df33_w = df_33.groupby("store_cd")[["amount"]].mean()
df33_w[(df33_w["amount"]>=330)]


amount
store_cd            
S12013    330.194130
S13001    348.470386
S13003    350.915519
S13004    330.943949
S13015    351.111960
S13019    330.208616
S13020    337.879932
S13052    402.867470
S14010    348.791262
S14011    335.718333
S14026    332.340588
S14045    330.082073
S14047    330.077073

In [8]:
type(df33_w)

pandas.core.frame.DataFrame

# P-34

---
> P-034: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。


In [3]:
df_34 = df_receipt.copy()


In [5]:
df_receipt[df_receipt['customer_id'].str.startswith("Z")]

sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
3        20190205   1549324800   S14042        1132               1   
8        20170504   1493856000   S13020        1112               2   
14       20180329   1522281600   S12014        1122               1   
15       20170116   1484524800   S13002        1112               1   
16       20190621   1561075200   S13044        1142               2   
...           ...          ...      ...         ...             ...   
104672   20180721   1532131200   S13043        1192               2   
104675   20190430   1556582400   S13041        1122               1   
104676   20180221   1519171200   S13043        1132               2   
104677   20190911   1568160000   S14047        1132               2   
104680   20190423   1555977600   S13016        1102               2   

           customer_id  product_cd  quantity  amount  
3       ZZ000000000000  P050301001         1      25  
8       ZZ000000000000  P071302010         1     770  
14      ZZ000000000000  P050104001         1     115  
15      ZZ000000000000  P080803001         1     100  
16      ZZ000000000000  P040102001         1     268  
...                ...         ...       ...     ...  
104672  ZZ000000000000  P059001015         1     308  
104675  ZZ000000000000  P070503012         1     100  
104676  ZZ000000000000  P050101001         1      40  
104677  ZZ000000000000  P071006005         1     218  
104680  ZZ000000000000  P050601001         1     138  

[38999 rows x 9 columns]

In [6]:
df34_c = df_receipt[~df_receipt['customer_id'].str.startswith("Z")]
df34_c


sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
0        20181103   1541203200   S14006         112               1   
1        20181118   1542499200   S13008        1132               2   
2        20170712   1499817600   S14028        1102               1   
4        20180821   1534809600   S14025        1102               2   
5        20190605   1559692800   S13003        1112               1   
...           ...          ...      ...         ...             ...   
104671   20180131   1517356800   S14010        1102               1   
104673   20181217   1545004800   S13004        1142               2   
104674   20190911   1568160000   S14046        1182               1   
104678   20170311   1489190400   S14040        1122               1   
104679   20170331   1490918400   S13002        1142               1   

           customer_id  product_cd  quantity  amount  
0       CS006214000001  P070305012         1     158  
1       CS008415000097  P070701017         1      81  
2       CS028414000014  P060101005         1     170  
4       CS025415000050  P060102007         1      90  
5       CS003515000195  P050102002         1     138  
...                ...         ...       ...     ...  
104671  CS010414000008  P060103003         1     150  
104673  CS004515000066  P059001016         1     308  
104674  CS046415000017  P070703003         1      98  
104678  CS040513000195  P050405003         1     168  
104679  CS002513000049  P060303001         1     148  

[65682 rows x 9 columns]

In [7]:
df34_c.groupby("customer_id")["amount"].sum()

customer_id
CS001113000004    1298
CS001114000005     626
CS001115000010    3044
CS001205000004    1988
CS001205000006    3337
                  ... 
CS051212000001     336
CS051513000004     551
CS051515000002     265
CS052212000002     192
CS052514000001     178
Name: amount, Length: 8306, dtype: int64

In [8]:
mean_val = df34_c.groupby("customer_id")["amount"].sum().mean()
mean_val

2547.742234529256

# P-35

---
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは10件だけ表示させれば良い。

In [9]:
df35 = df_receipt.copy()
df35_c = df35[~df35['customer_id'].str.startswith("Z")].groupby("customer_id")["amount"].sum()

In [10]:
df35_c[df35_c>=df35_c.mean()].head(10)

customer_id
CS001115000010    3044
CS001205000006    3337
CS001214000009    4685
CS001214000017    4132
CS001214000052    5639
CS001215000040    3496
CS001304000006    3726
CS001305000005    3485
CS001305000011    4370
CS001315000180    3300
Name: amount, dtype: int64

# P-36

---
> P-036: レシート明細データフレーム（df_receipt）と店舗データフレーム（df_store）を内部結合し、レシート明細データフレームの全項目と店舗データフレームの店舗名（store_name）を10件表示させよ。

In [11]:
df36_receipt = df_receipt.copy()
df36_store = df_store.copy()

In [12]:
df36_receipt.head(10)

sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
0   20181103   1541203200   S14006         112               1   
1   20181118   1542499200   S13008        1132               2   
2   20170712   1499817600   S14028        1102               1   
3   20190205   1549324800   S14042        1132               1   
4   20180821   1534809600   S14025        1102               2   
5   20190605   1559692800   S13003        1112               1   
6   20181205   1543968000   S14024        1102               2   
7   20190922   1569110400   S14040        1102               1   
8   20170504   1493856000   S13020        1112               2   
9   20191010   1570665600   S14027        1102               1   

      customer_id  product_cd  quantity  amount  
0  CS006214000001  P070305012         1     158  
1  CS008415000097  P070701017         1      81  
2  CS028414000014  P060101005         1     170  
3  ZZ000000000000  P050301001         1      25  
4  CS025415000050  P060102007         1      90  
5  CS003515000195  P050102002         1     138  
6  CS024514000042  P080101005         1      30  
7  CS040415000178  P070501004         1     128  
8  ZZ000000000000  P071302010         1     770  
9  CS027514000015  P071101003         1     680

In [13]:
df36_store.head(10)

store_cd store_name  prefecture_cd prefecture              address  \
0   S12014       千草台店             12        千葉県      千葉県千葉市稲毛区千草台一丁目   
1   S13002       国分寺店             13        東京都         東京都国分寺市本多二丁目   
2   S14010        菊名店             14       神奈川県      神奈川県横浜市港北区菊名一丁目   
3   S14033       阿久和店             14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
4   S14036     相模原中央店             14       神奈川県        神奈川県相模原市中央二丁目   
5   S13051      板橋大原店             13        東京都            東京都板橋区大原町   
6   S13015        南砂店             13        東京都          東京都江東区南砂二丁目   
7   S14040       長津田店             14       神奈川県  神奈川県横浜市緑区長津田みなみ台五丁目   
8   S13044       南六郷店             13        東京都         東京都大田区南六郷二丁目   
9   S14050      阿久和西店             14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   

                    address_kana        tel_no  longitude  latitude  \
0         チバケンチバシイナゲクチグサダイイッチョウメ  043-123-4003   140.1180  35.63559   
1           トウキョウトコクブンジシホンダニチョウメ  042-123-4008   139.4802  35.70566   
2      カナガワケンヨコハマシコウホククキクナイッチョウメ  045-123-4032   139.6326  35.50049   
3      カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4043   139.4961  35.45918   
4         カナガワケンサガミハラシチュウオウニチョウメ  042-123-4045   139.3716  35.57327   
5             トウキョウトイタバシクオオハラチョウ  03-0123-4029   139.6980  35.76788   
6          トウキョウトコウトウクミナミスナニチョウメ  03-0123-4014   139.8229  35.67066   
7  カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ  045-123-4046   139.4994  35.52398   
8         トウキョウトオオタクミナミロクゴウニチョウメ  03-0123-4028   139.7207  35.54604   
9      カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4053   139.4961  35.45918   

   floor_area  
0      1698.0  
1      1735.0  
2      1732.0  
3      1495.0  
4      1679.0  
5      1045.0  
6      1337.0  
7      1548.0  
8      1379.0  
9      1830.0

In [14]:
pd.merge(df36_receipt,df36_store[["store_cd","store_name"]],on="store_cd").head(10)

sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
0   20181103   1541203200   S14006         112               1   
1   20181116   1542326400   S14006         112               2   
2   20170118   1484697600   S14006        1162               1   
3   20190524   1558656000   S14006        1192               1   
4   20190419   1555632000   S14006         112               2   
5   20181119   1542585600   S14006        1152               2   
6   20171211   1512950400   S14006        1132               2   
7   20191021   1571616000   S14006        1112               2   
8   20170710   1499644800   S14006        1132               2   
9   20190805   1564963200   S14006         112               1   

      customer_id  product_cd  quantity  amount store_name  
0  CS006214000001  P070305012         1     158       葛が谷店  
1  ZZ000000000000  P080401001         1      48       葛が谷店  
2  CS006815000006  P050406035         1     220       葛が谷店  
3  CS006514000034  P060104003         1      80       葛が谷店  
4  ZZ000000000000  P060501002         1     148       葛が谷店  
5  ZZ000000000000  P050701001         1      88       葛が谷店  
6  CS006515000175  P090903001         1      80       葛が谷店  
7  CS006415000221  P040602001         1     405       葛が谷店  
8  CS006411000036  P090301051         1     330       葛が谷店  
9  CS006211000012  P050104001         1     115       葛が谷店

# P-37

---
> P-037: 商品データフレーム（df_product）とカテゴリデータフレーム（df_category）を内部結合し、商品データフレームの全項目とカテゴリデータフレームの小区分名（category_small_name）を10件表示させよ。

In [15]:
df37_p=df_product.copy()
df37_c=df_category.copy()
df37_p.head()

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040101001                  4                 401              40101   
1  P040101002                  4                 401              40101   
2  P040101003                  4                 401              40101   
3  P040101004                  4                 401              40101   
4  P040101005                  4                 401              40101   

   unit_price  unit_cost  
0       198.0      149.0  
1       218.0      164.0  
2       230.0      173.0  
3       248.0      186.0  
4       268.0      201.0

In [16]:
df37_c.head()

category_major_cd category_major_name  category_medium_cd  \
0                  4                  惣菜                 401   
1                  4                  惣菜                 401   
2                  4                  惣菜                 402   
3                  4                  惣菜                 402   
4                  4                  惣菜                 402   

  category_medium_name  category_small_cd category_small_name  
0                  御飯類              40101                 弁当類  
1                  御飯類              40102                 寿司類  
2                  佃煮類              40201               魚介佃煮類  
3                  佃煮類              40202               海草佃煮類  
4                  佃煮類              40203               野菜佃煮類

In [17]:
pd.merge(df37_p,df37_c[["category_small_cd","category_small_name"]],on="category_small_cd").head(10)

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040101001                  4                 401              40101   
1  P040101002                  4                 401              40101   
2  P040101003                  4                 401              40101   
3  P040101004                  4                 401              40101   
4  P040101005                  4                 401              40101   
5  P040101006                  4                 401              40101   
6  P040101007                  4                 401              40101   
7  P040101008                  4                 401              40101   
8  P040101009                  4                 401              40101   
9  P040101010                  4                 401              40101   

   unit_price  unit_cost category_small_name  
0       198.0      149.0                 弁当類  
1       218.0      164.0                 弁当類  
2       230.0      173.0                 弁当類  
3       248.0      186.0                 弁当類  
4       268.0      201.0                 弁当類  
5       298.0      224.0                 弁当類  
6       338.0      254.0                 弁当類  
7       420.0      315.0                 弁当類  
8       498.0      374.0                 弁当類  
9       580.0      435.0                 弁当類

# P-38

---
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は10件だけ表示させれば良い。

In [18]:
df38_r = df_receipt.copy()
df38_c = df_customer.copy()


In [19]:
df38_c['customer_id'].str.startswith("Z")

0        False
1        False
2        False
3        False
4        False
         ...  
21966    False
21967    False
21968    False
21969    False
21970    False
Name: customer_id, Length: 21971, dtype: bool

In [14]:
df38_r['customer_id'].str.startswith("Z")

0         False
1         False
2         False
3          True
4         False
          ...  
104676     True
104677     True
104678    False
104679    False
104680     True
Name: customer_id, Length: 104681, dtype: bool

In [20]:
df38_c[df38_c['customer_id'].str.startswith("Z")]

Empty DataFrame
Columns: [customer_id, customer_name, gender_cd, gender, birth_day, age, postal_cd, address, application_store_cd, application_date, status_cd]
Index: []

In [21]:
df38_r[df38_r['customer_id'].str.startswith("Z")]

sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
3        20190205   1549324800   S14042        1132               1   
8        20170504   1493856000   S13020        1112               2   
14       20180329   1522281600   S12014        1122               1   
15       20170116   1484524800   S13002        1112               1   
16       20190621   1561075200   S13044        1142               2   
...           ...          ...      ...         ...             ...   
104672   20180721   1532131200   S13043        1192               2   
104675   20190430   1556582400   S13041        1122               1   
104676   20180221   1519171200   S13043        1132               2   
104677   20190911   1568160000   S14047        1132               2   
104680   20190423   1555977600   S13016        1102               2   

           customer_id  product_cd  quantity  amount  
3       ZZ000000000000  P050301001         1      25  
8       ZZ000000000000  P071302010         1     770  
14      ZZ000000000000  P050104001         1     115  
15      ZZ000000000000  P080803001         1     100  
16      ZZ000000000000  P040102001         1     268  
...                ...         ...       ...     ...  
104672  ZZ000000000000  P059001015         1     308  
104675  ZZ000000000000  P070503012         1     100  
104676  ZZ000000000000  P050101001         1      40  
104677  ZZ000000000000  P071006005         1     218  
104680  ZZ000000000000  P050601001         1     138  

[38999 rows x 9 columns]

In [22]:
df38_c2 = df38_c[(~df38_c['customer_id'].str.startswith("Z")) & (df38_c["gender_cd"] == 1)]


In [23]:
df38_c2

customer_id customer_name  gender_cd gender   birth_day  age  \
0      CS021313000114        大野 あや子          1     女性  1981-04-29   37   
2      CS031415000172       宇多田 貴美子          1     女性  1976-10-04   42   
3      CS028811000001        堀井 かおり          1     女性  1933-03-27   86   
4      CS001215000145         田崎 美紀          1     女性  1995-03-29   24   
6      CS015414000103         奥野 陽子          1     女性  1977-08-09   41   
...               ...           ...        ...    ...         ...  ...   
21965  CS042513000030        三宅 ヒカル          1     女性  1959-01-19   60   
21966  CS002512000474         市村 夏希          1     女性  1959-10-12   59   
21967  CS029414000065         上村 怜奈          1     女性  1970-10-19   48   
21969  CS033512000184         池谷 華子          1     女性  1964-06-05   54   
21970  CS009213000022          稲垣 咲          1     女性  1996-08-16   22   

      postal_cd                 address application_store_cd  \
0      259-1113    神奈川県伊勢原市粟窪**********               S14021   
2      151-0053     東京都渋谷区代々木**********               S13031   
3      245-0016  神奈川県横浜市泉区和泉町**********               S14028   
4      144-0055     東京都大田区仲六郷**********               S13001   
6      136-0073      東京都江東区北砂**********               S13015   
...         ...                     ...                  ...   
21965  231-0865  神奈川県横浜市中区北方町**********               S14042   
21966  185-0034     東京都国分寺市光町**********               S13002   
21967  279-0043     千葉県浦安市富士見**********               S12029   
21969  245-0016  神奈川県横浜市泉区和泉町**********               S14033   
21970  154-0012     東京都世田谷区駒沢**********               S13009   

       application_date     status_cd  
0              20150905  0-00000000-0  
2              20150529  D-20100325-C  
3              20160115  0-00000000-0  
4              20170605  6-20090929-2  
6              20150722  B-20100609-B  
...                 ...           ...  
21965          20181114  0-00000000-0  
21966          20171110  0-00000000-0  
21967          20150313  F-20101028-F  
21969          20160206  0-00000000-0  
21970          20150424  0-00000000-0  

[17918 rows x 11 columns]

In [31]:
df38_r2 = df38_r[(~df38_r['customer_id'].str.startswith("Z"))].groupby("customer_id")["amount"].sum()

In [32]:
df38_r2

customer_id
CS001113000004    1298
CS001114000005     626
CS001115000010    3044
CS001205000004    1988
CS001205000006    3337
                  ... 
CS051212000001     336
CS051513000004     551
CS051515000002     265
CS052212000002     192
CS052514000001     178
Name: amount, Length: 8306, dtype: int64

In [33]:
pd.merge(df38_c2,df38_r2,how="left",on="customer_id").fillna(0).head(10)

customer_id customer_name  gender_cd gender   birth_day  age postal_cd  \
0  CS021313000114        大野 あや子          1     女性  1981-04-29   37  259-1113   
1  CS031415000172       宇多田 貴美子          1     女性  1976-10-04   42  151-0053   
2  CS028811000001        堀井 かおり          1     女性  1933-03-27   86  245-0016   
3  CS001215000145         田崎 美紀          1     女性  1995-03-29   24  144-0055   
4  CS015414000103         奥野 陽子          1     女性  1977-08-09   41  136-0073   
5  CS033513000180          安斎 遥          1     女性  1962-07-11   56  241-0823   
6  CS035614000014        板倉 菜々美          1     女性  1954-07-16   64  154-0015   
7  CS011215000048         芦田 沙耶          1     女性  1992-02-01   27  223-0062   
8  CS009413000079         市川 コウ          1     女性  1975-12-28   43  158-0093   
9  CS040412000191         川井 郁恵          1     女性  1977-01-05   42  226-0021   

                    address application_store_cd  application_date  \
0      神奈川県伊勢原市粟窪**********               S14021          20150905   
1       東京都渋谷区代々木**********               S13031          20150529   
2    神奈川県横浜市泉区和泉町**********               S14028          20160115   
3       東京都大田区仲六郷**********               S13001          20170605   
4        東京都江東区北砂**********               S13015          20150722   
5    神奈川県横浜市旭区善部町**********               S14033          20150728   
6      東京都世田谷区桜新町**********               S13035          20150804   
7  神奈川県横浜市港北区日吉本町**********               S14011          20150228   
8      東京都世田谷区上野毛**********               S13009          20151209   
9   神奈川県横浜市緑区北八朔町**********               S14040          20151101   

      status_cd  amount  
0  0-00000000-0     0.0  
1  D-20100325-C  5088.0  
2  0-00000000-0     0.0  
3  6-20090929-2   875.0  
4  B-20100609-B  3122.0  
5  6-20080506-5   868.0  
6  0-00000000-0     0.0  
7  C-20100421-9  3444.0  
8  0-00000000-0     0.0  
9  1-20091025-4   210.0

# P-39

---
> P-039: レシート明細データフレーム（df_receipt）から売上日数の多い顧客の上位20件と、売上金額合計の多い顧客の上位20件を抽出し、完全外部結合せよ。ただし、非会員（顧客IDが'Z'から始まるもの）は除外すること。

In [142]:
df39_r = df_receipt.copy()

In [146]:
df39_day_cnt= df39_r[~df39_r["customer_id"].str.startswith("Z")].groupby("customer_id")["sales_ymd"].count().sort_values(ascending=False).head(20)

In [147]:
df39_day_cnt

customer_id
CS040214000008    46
CS010214000010    44
CS015415000185    44
CS028415000007    42
CS010214000002    42
CS021514000045    40
CS007515000107    40
CS017415000097    40
CS016415000141    40
CS022515000226    38
CS021515000172    38
CS032415000209    38
CS031414000051    38
CS031414000073    38
CS014214000023    38
CS039414000052    38
CS014415000077    38
CS021515000101    36
CS021514000008    36
CS022515000028    36
Name: sales_ymd, dtype: int64

In [148]:
df39_sales_amount = df39_r[~df39_r["customer_id"].str.startswith("Z")].groupby("customer_id")["amount"].sum().sort_values(ascending=False).head(20)

In [149]:
pd.merge(df39_day_cnt,df39_sales_amount,how="outer",on="customer_id").fillna(0)

sales_ymd   amount
customer_id                       
CS040214000008       46.0      0.0
CS010214000010       44.0  18585.0
CS015415000185       44.0  20153.0
CS028415000007       42.0  19127.0
CS010214000002       42.0      0.0
CS021514000045       40.0      0.0
CS007515000107       40.0      0.0
CS017415000097       40.0  23086.0
CS016415000141       40.0  18372.0
CS022515000226       38.0      0.0
CS021515000172       38.0      0.0
CS032415000209       38.0      0.0
CS031414000051       38.0  19202.0
CS031414000073       38.0      0.0
CS014214000023       38.0      0.0
CS039414000052       38.0      0.0
CS014415000077       38.0      0.0
CS021515000101       36.0      0.0
CS021514000008       36.0      0.0
CS022515000028       36.0      0.0
CS001605000009        0.0  18925.0
CS006515000023        0.0  18372.0
CS011414000106        0.0  18338.0
CS038415000104        0.0  17847.0
CS035414000024        0.0  17615.0
CS021515000089        0.0  17580.0
CS032414000072        0.0  16563.0
CS016415000101        0.0  16348.0
CS011415000006        0.0  16094.0
CS034415000047        0.0  16083.0
CS007514000094        0.0  15735.0
CS009414000059        0.0  15492.0
CS030415000034        0.0  15468.0
CS015515000034        0.0  15300.0

# P-40

---
> P-040: 全ての店舗と全ての商品を組み合わせると何件のデータとなるか調査したい。店舗（df_store）と商品（df_product）を直積した件数を計算せよ。

In [50]:
#df_storeの店舗の件数とdf_productの商品の件数を掛け合わせる
len(df_store["store_cd"].unique()) * len(df_product["product_cd"].unique())

531590

# P-41

---
> P-041: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前日からの売上金額増減を計算せよ。なお、計算結果は10件表示すればよい。

In [51]:
df_sales_amount_by_date = df_receipt[['sales_ymd', 'amount']].groupby('sales_ymd').sum().reset_index()
df_sales_amount_by_date = pd.concat([df_sales_amount_by_date, df_sales_amount_by_date.shift()], axis=1)
df_sales_amount_by_date.columns = ['sales_ymd','amount','lag_ymd','lag_amount']
df_sales_amount_by_date['diff_amount'] = df_sales_amount_by_date['amount'] - df_sales_amount_by_date['lag_amount']
df_sales_amount_by_date.head(10)

sales_ymd  amount     lag_ymd  lag_amount  diff_amount
0   20170101   33723         NaN         NaN          NaN
1   20170102   24165  20170101.0     33723.0      -9558.0
2   20170103   27503  20170102.0     24165.0       3338.0
3   20170104   36165  20170103.0     27503.0       8662.0
4   20170105   37830  20170104.0     36165.0       1665.0
5   20170106   32387  20170105.0     37830.0      -5443.0
6   20170107   23415  20170106.0     32387.0      -8972.0
7   20170108   24737  20170107.0     23415.0       1322.0
8   20170109   26718  20170108.0     24737.0       1981.0
9   20170110   20143  20170109.0     26718.0      -6575.0

# P-42

---
> P-042: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、１日前、２日前、３日前のデータを結合せよ。結果は10件表示すればよい。

In [150]:
df42_sum = df_receipt[['sales_ymd', 'amount']].groupby('sales_ymd').sum()
df42_sum

amount
sales_ymd        
20170101    33723
20170102    24165
20170103    27503
20170104    36165
20170105    37830
...           ...
20191027    37484
20191028    40161
20191029    36091
20191030    26602
20191031    25216

[1034 rows x 1 columns]

In [151]:
df42_sum["1dayAgo"] = df42_sum["amount"].shift(1)
df42_sum["2dayAgo"] = df42_sum["amount"].shift(2)
df42_sum["3dayAgo"] = df42_sum["amount"].shift(3)
df42_sum.head(10)



amount  1dayAgo  2dayAgo  3dayAgo
sales_ymd                                   
20170101    33723      NaN      NaN      NaN
20170102    24165  33723.0      NaN      NaN
20170103    27503  24165.0  33723.0      NaN
20170104    36165  27503.0  24165.0  33723.0
20170105    37830  36165.0  27503.0  24165.0
20170106    32387  37830.0  36165.0  27503.0
20170107    23415  32387.0  37830.0  36165.0
20170108    24737  23415.0  32387.0  37830.0
20170109    26718  24737.0  23415.0  32387.0
20170110    20143  26718.0  24737.0  23415.0

# P-43

---
> P-043： レシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合し、性別（gender）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータフレーム（df_sales_summary）を作成せよ。性別は0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [175]:
df_tmp = pd.merge(df_receipt, df_customer, how ='inner', on="customer_id")
df_tmp['area'] = df_tmp['age'].apply(lambda x: math.floor(x / 10) * 10)
df_tmp

sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
0       20181103   1541203200   S14006         112               1   
1       20170509   1494288000   S14006         112               1   
2       20170608   1496880000   S14006         112               1   
3       20170608   1496880000   S14006         112               2   
4       20181028   1540684800   S14006         112               2   
...          ...          ...      ...         ...             ...   
65677   20171202   1512172800   S13004        1152               1   
65678   20180421   1524268800   S13002        1142               2   
65679   20180421   1524268800   S13002        1142               1   
65680   20190416   1555372800   S14040        1182               2   
65681   20190416   1555372800   S14040        1182               1   

          customer_id  product_cd  quantity  amount customer_name  gender_cd  \
0      CS006214000001  P070305012         1     158         志水 佳乃          1   
1      CS006214000001  P071401004         1    1100         志水 佳乃          1   
2      CS006214000001  P060104021         1     120         志水 佳乃          1   
3      CS006214000001  P080403001         1     175         志水 佳乃          1   
4      CS006214000001  P050102004         1     188         志水 佳乃          1   
...               ...         ...       ...     ...           ...        ...   
65677  CS004613000146  P071302002         1     308         玉木 恵麻          1   
65678  CS002314000037  P070703051         1     228          島崎 愛          1   
65679  CS002314000037  P060702014         1     108          島崎 愛          1   
65680  CS040311000022  P071401017         1    2200        五十嵐 南朋          1   
65681  CS040311000022  P071101020         1     980        五十嵐 南朋          1   

      gender   birth_day  age postal_cd                  address  \
0         女性  1996-12-08   22  224-0057  神奈川県横浜市都筑区川和町**********   
1         女性  1996-12-08   22  224-0057  神奈川県横浜市都筑区川和町**********   
2         女性  1996-12-08   22  224-0057  神奈川県横浜市都筑区川和町**********   
3         女性  1996-12-08   22  224-0057  神奈川県横浜市都筑区川和町**********   
4         女性  1996-12-08   22  224-0057  神奈川県横浜市都筑区川和町**********   
...      ...         ...  ...       ...                      ...   
65677     女性  1953-02-01   66  165-0032       東京都中野区鷺宮**********   
65678     女性  1979-01-07   40  185-0012      東京都国分寺市本町**********   
65679     女性  1979-01-07   40  185-0012      東京都国分寺市本町**********   
65680     女性  1986-08-11   32  194-0014      東京都町田市高ヶ坂**********   
65681     女性  1986-08-11   32  194-0014      東京都町田市高ヶ坂**********   

      application_store_cd  application_date     status_cd  area  
0                   S14006          20150201  E-20100908-F    20  
1                   S14006          20150201  E-20100908-F    20  
2                   S14006          20150201  E-20100908-F    20  
3                   S14006          20150201  E-20100908-F    20  
4                   S14006          20150201  E-20100908-F    20  
...                    ...               ...           ...   ...  
65677               S13004          20160813  4-20081202-1    60  
65678               S13002          20151202  2-20090421-1    40  
65679               S13002          20151202  2-20090421-1    40  
65680               S14040          20150419  B-20100416-4    30  
65681               S14040          20150419  B-20100416-4    30  

[65682 rows x 20 columns]

In [174]:
df_sales_summary = pd.pivot_table(df_tmp, index='area', columns='gender_cd', values='amount', aggfunc='sum')
df_sales_summary

gender_cd         0          1         9
area                                    
10           1591.0   149836.0    4317.0
20          72940.0  1363724.0   44328.0
30         177322.0   693047.0   50441.0
40          19355.0  9320791.0  483512.0
50          54320.0  6685192.0  342923.0
60         272469.0   987741.0   71418.0
70          13435.0    29764.0    2427.0
80          46360.0   262923.0    5111.0
90              NaN     6260.0       NaN

In [170]:
#たろうさんの回答
df_sales_summary = df_customer.merge(df_receipt, left_on="customer_id", right_on="customer_id", how="outer")
pd.cut(df_sales_summary["age"],range(-1,100,10)[1:])



0         (29.0, 39.0]
1         (59.0, 69.0]
2         (39.0, 49.0]
3         (39.0, 49.0]
4         (39.0, 49.0]
              ...     
118341             NaN
118342             NaN
118343             NaN
118344             NaN
118345             NaN
Name: age, Length: 118346, dtype: category
Categories (9, interval[int64]): [(9, 19] < (19, 29] < (29, 39] < (39, 49] ... (59, 69] < (69, 79] < (79, 89] < (89, 99]]

In [171]:
#9<age<=19を10第で考えて、はばをもたせた枠をつくってる
df_sales_summary = df_sales_summary.groupby(by=[pd.cut(df_sales_summary["age"],range(-1,100,10)[1:]), "gender_cd"])["amount"].sum()
df_sales_summary

age       gender_cd
(9, 19]   0.0             1591.0
          1.0           149836.0
          9.0             4317.0
(19, 29]  0.0            72940.0
          1.0          1363724.0
          9.0            44328.0
(29, 39]  0.0           177322.0
          1.0           693047.0
          9.0            50441.0
(39, 49]  0.0            19355.0
          1.0          9320791.0
          9.0           483512.0
(49, 59]  0.0            54320.0
          1.0          6685192.0
          9.0           342923.0
(59, 69]  0.0           272469.0
          1.0           987741.0
          9.0            71418.0
(69, 79]  0.0            13435.0
          1.0            29764.0
          9.0             2427.0
(79, 89]  0.0            46360.0
          1.0           262923.0
          9.0             5111.0
(89, 99]  0.0                0.0
          1.0             6260.0
          9.0                NaN
Name: amount, dtype: float64

In [172]:
df_sales_summary.unstack()

gender_cd       0.0        1.0       9.0
age                                     
(9, 19]      1591.0   149836.0    4317.0
(19, 29]    72940.0  1363724.0   44328.0
(29, 39]   177322.0   693047.0   50441.0
(39, 49]    19355.0  9320791.0  483512.0
(49, 59]    54320.0  6685192.0  342923.0
(59, 69]   272469.0   987741.0   71418.0
(69, 79]    13435.0    29764.0    2427.0
(79, 89]    46360.0   262923.0    5111.0
(89, 99]        0.0     6260.0       NaN

In [173]:
list(range(-1,100,10)[1:])

[9, 19, 29, 39, 49, 59, 69, 79, 89, 99]

# P-44

---
> P-044： 前設問で作成した売上サマリデータフレーム（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータフレームから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を'00'、女性を'01'、不明を'99'とする。

In [176]:
#T：転置　縦と横をひっくり返すこと、よく使う
df_sales_summary.T

area             10         20        30         40         50        60  \
gender_cd                                                                  
0            1591.0    72940.0  177322.0    19355.0    54320.0  272469.0   
1          149836.0  1363724.0  693047.0  9320791.0  6685192.0  987741.0   
9            4317.0    44328.0   50441.0   483512.0   342923.0   71418.0   

area            70        80      90  
gender_cd                             
0          13435.0   46360.0     NaN  
1          29764.0  262923.0  6260.0  
9           2427.0    5111.0     NaN

# P-45

---
> P-045: 顧客データフレーム（df_customer）の生年月日（birth_day）は日付型（Date）でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [72]:
df45 = df_customer[["customer_id","birth_day"]]

In [90]:
df45["birth_day"] = pd.to_datetime(df45['birth_day']).dt.strftime("%Y%m%d")

In [92]:
df45.head(10)

customer_id birth_day
0  CS021313000114  19810429
1  CS037613000071  19520401
2  CS031415000172  19761004
3  CS028811000001  19330327
4  CS001215000145  19950329
5  CS020401000016  19740915
6  CS015414000103  19770809
7  CS029403000008  19730817
8  CS015804000004  19310502
9  CS033513000180  19620711

# P-46

---
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [29]:
df46 = df_customer[["customer_id","application_date"]]
df46[["application_date"]]

application_date
0              20150905
1              20150414
2              20150529
3              20160115
4              20170605
...                 ...
21966          20171110
21967          20150313
21968          20150406
21969          20160206
21970          20150424

[21971 rows x 1 columns]

---
> P-047: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [7]:
df46["application_date"] = pd.to_datetime(df46["application_date"], format='%Y%m%d')
df46["application_date"]

0       2015-09-05
1       2015-04-14
2       2015-05-29
3       2016-01-15
4       2017-06-05
           ...    
21966   2017-11-10
21967   2015-03-13
21968   2015-04-06
21969   2016-02-06
21970   2015-04-24
Name: application_date, Length: 21971, dtype: datetime64[ns]

In [9]:
df46.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21971 entries, 0 to 21970
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   customer_id       21971 non-null  object        
 1   application_date  21971 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 343.4+ KB


In [101]:
df47 = df_receipt[["sales_ymd","receipt_no","receipt_sub_no"]]
df47

sales_ymd  receipt_no  receipt_sub_no
0        20181103         112               1
1        20181118        1132               2
2        20170712        1102               1
3        20190205        1132               1
4        20180821        1102               2
...           ...         ...             ...
104676   20180221        1132               2
104677   20190911        1132               2
104678   20170311        1122               1
104679   20170331        1142               1
104680   20190423        1102               2

[104681 rows x 3 columns]

In [103]:
df47["sales_ymd"] = pd.to_datetime(df47["sales_ymd"], format='%Y%m%d')
df47

sales_ymd  receipt_no  receipt_sub_no
0      2018-11-03         112               1
1      2018-11-18        1132               2
2      2017-07-12        1102               1
3      2019-02-05        1132               1
4      2018-08-21        1102               2
...           ...         ...             ...
104676 2018-02-21        1132               2
104677 2019-09-11        1132               2
104678 2017-03-11        1122               1
104679 2017-03-31        1142               1
104680 2019-04-23        1102               2

[104681 rows x 3 columns]

# P-48

---
> P-048: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [13]:
df48 = df_receipt[["sales_epoch","receipt_no","receipt_sub_no"]]
df48

sales_epoch  receipt_no  receipt_sub_no
0        1541203200         112               1
1        1542499200        1132               2
2        1499817600        1102               1
3        1549324800        1132               1
4        1534809600        1102               2
...             ...         ...             ...
104676   1519171200        1132               2
104677   1568160000        1132               2
104678   1489190400        1122               1
104679   1490918400        1142               1
104680   1555977600        1102               2

[104681 rows x 3 columns]

In [14]:
df48["sales_epoch"] = pd.to_datetime(df48["sales_epoch"].astype(int), unit='s')
df48.head(10)

sales_epoch  receipt_no  receipt_sub_no
0  2018-11-03         112               1
1  2018-11-18        1132               2
2  2017-07-12        1102               1
3  2019-02-05        1132               1
4  2018-08-21        1102               2
5  2019-06-05        1112               1
6  2018-12-05        1102               2
7  2019-09-22        1102               1
8  2017-05-04        1112               2
9  2019-10-10        1102               1

# P-49

---
> P-049: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"年"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [128]:
df49 = df48.copy()
df49

sales_epoch  receipt_no  receipt_sub_no
0       2018-11-03         112               1
1       2018-11-18        1132               2
2       2017-07-12        1102               1
3       2019-02-05        1132               1
4       2018-08-21        1102               2
...            ...         ...             ...
104676  2018-02-21        1132               2
104677  2019-09-11        1132               2
104678  2017-03-11        1122               1
104679  2017-03-31        1142               1
104680  2019-04-23        1102               2

[104681 rows x 3 columns]

In [129]:
df49.dtypes


sales_epoch       datetime64[ns]
receipt_no                 int64
receipt_sub_no             int64
dtype: object

In [130]:
df49["sales_epoch_Year"] =df49["sales_epoch"].dt.strftime("%Y")
df49[["sales_epoch_Year","receipt_no","receipt_sub_no"]].head(10)


sales_epoch_Year  receipt_no  receipt_sub_no
0             2018         112               1
1             2018        1132               2
2             2017        1102               1
3             2019        1132               1
4             2018        1102               2
5             2019        1112               1
6             2018        1102               2
7             2019        1102               1
8             2017        1112               2
9             2019        1102               1

# P-50

---
> P-050: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"月"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"月"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [131]:
df50= df48.copy()
df50

sales_epoch  receipt_no  receipt_sub_no
0       2018-11-03         112               1
1       2018-11-18        1132               2
2       2017-07-12        1102               1
3       2019-02-05        1132               1
4       2018-08-21        1102               2
...            ...         ...             ...
104676  2018-02-21        1132               2
104677  2019-09-11        1132               2
104678  2017-03-11        1122               1
104679  2017-03-31        1142               1
104680  2019-04-23        1102               2

[104681 rows x 3 columns]

In [134]:
df50["sales_epoch_MM"] = df50["sales_epoch"].dt.strftime("%m")
df50[["sales_epoch_MM","receipt_no","receipt_sub_no"]].head(10)

sales_epoch_MM  receipt_no  receipt_sub_no
0             11         112               1
1             11        1132               2
2             07        1102               1
3             02        1132               1
4             08        1102               2
5             06        1112               1
6             12        1102               2
7             09        1102               1
8             05        1112               2
9             10        1102               1

# P-51

---
> P-051: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"日"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"日"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [135]:
df51= df48.copy()
df51

sales_epoch  receipt_no  receipt_sub_no
0       2018-11-03         112               1
1       2018-11-18        1132               2
2       2017-07-12        1102               1
3       2019-02-05        1132               1
4       2018-08-21        1102               2
...            ...         ...             ...
104676  2018-02-21        1132               2
104677  2019-09-11        1132               2
104678  2017-03-11        1122               1
104679  2017-03-31        1142               1
104680  2019-04-23        1102               2

[104681 rows x 3 columns]

In [136]:
df51["sales_epoch_DD"] = df50["sales_epoch"].dt.strftime("%d")
df51[["sales_epoch_DD","receipt_no","receipt_sub_no"]].head(10)

sales_epoch_DD  receipt_no  receipt_sub_no
0             03         112               1
1             18        1132               2
2             12        1102               1
3             05        1132               1
4             21        1102               2
5             05        1112               1
6             05        1102               2
7             22        1102               1
8             04        1112               2
9             10        1102               1

# P-52

---
> P-052: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2000円以下を0、2000円超を1に2値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [15]:
df52 = df_receipt.copy()
df52

sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
0        20181103   1541203200   S14006         112               1   
1        20181118   1542499200   S13008        1132               2   
2        20170712   1499817600   S14028        1102               1   
3        20190205   1549324800   S14042        1132               1   
4        20180821   1534809600   S14025        1102               2   
...           ...          ...      ...         ...             ...   
104676   20180221   1519171200   S13043        1132               2   
104677   20190911   1568160000   S14047        1132               2   
104678   20170311   1489190400   S14040        1122               1   
104679   20170331   1490918400   S13002        1142               1   
104680   20190423   1555977600   S13016        1102               2   

           customer_id  product_cd  quantity  amount  
0       CS006214000001  P070305012         1     158  
1       CS008415000097  P070701017         1      81  
2       CS028414000014  P060101005         1     170  
3       ZZ000000000000  P050301001         1      25  
4       CS025415000050  P060102007         1      90  
...                ...         ...       ...     ...  
104676  ZZ000000000000  P050101001         1      40  
104677  ZZ000000000000  P071006005         1     218  
104678  CS040513000195  P050405003         1     168  
104679  CS002513000049  P060303001         1     148  
104680  ZZ000000000000  P050601001         1     138  

[104681 rows x 9 columns]

In [26]:
df52_amount = df52[(~df52["customer_id"].str.startswith("Z"))].groupby("customer_id")[["amount"]].sum()
df52_amount

amount
customer_id           
CS001113000004    1298
CS001114000005     626
CS001115000010    3044
CS001205000004    1988
CS001205000006    3337
...                ...
CS051212000001     336
CS051513000004     551
CS051515000002     265
CS052212000002     192
CS052514000001     178

[8306 rows x 1 columns]

In [31]:
def func_setkbn(x):
    ret = 0
    if x <= 2000:
        ret = 0
    else:
        ret = 1
    return ret
df52_amount["2000Over"] = df52_amount["amount"].apply(func_setkbn)
df52_amount.head(10)

amount  2000Over
customer_id                     
CS001113000004    1298         0
CS001114000005     626         0
CS001115000010    3044         1
CS001205000004    1988         0
CS001205000006    3337         1
CS001211000025     456         0
CS001212000027     448         0
CS001212000031     296         0
CS001212000046     228         0
CS001212000070     456         0

# P-53

---
> P-053: 顧客データフレーム（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に２値化せよ。さらにレシート明細データフレーム（df_receipt）と結合し、全期間において買い物実績のある顧客数を、作成した2値ごとにカウントせよ。

In [16]:
df53 = df_customer.copy()
df53

customer_id customer_name  gender_cd gender   birth_day  age  \
0      CS021313000114        大野 あや子          1     女性  1981-04-29   37   
1      CS037613000071         六角 雅彦          9     不明  1952-04-01   66   
2      CS031415000172       宇多田 貴美子          1     女性  1976-10-04   42   
3      CS028811000001        堀井 かおり          1     女性  1933-03-27   86   
4      CS001215000145         田崎 美紀          1     女性  1995-03-29   24   
...               ...           ...        ...    ...         ...  ...   
21966  CS002512000474         市村 夏希          1     女性  1959-10-12   59   
21967  CS029414000065         上村 怜奈          1     女性  1970-10-19   48   
21968  CS012403000043          堀越 明          0     男性  1972-12-16   46   
21969  CS033512000184         池谷 華子          1     女性  1964-06-05   54   
21970  CS009213000022          稲垣 咲          1     女性  1996-08-16   22   

      postal_cd                  address application_store_cd  \
0      259-1113     神奈川県伊勢原市粟窪**********               S14021   
1      136-0076       東京都江東区南砂**********               S13037   
2      151-0053      東京都渋谷区代々木**********               S13031   
3      245-0016   神奈川県横浜市泉区和泉町**********               S14028   
4      144-0055      東京都大田区仲六郷**********               S13001   
...         ...                      ...                  ...   
21966  185-0034      東京都国分寺市光町**********               S13002   
21967  279-0043      千葉県浦安市富士見**********               S12029   
21968  231-0825  神奈川県横浜市中区本牧間門**********               S14012   
21969  245-0016   神奈川県横浜市泉区和泉町**********               S14033   
21970  154-0012      東京都世田谷区駒沢**********               S13009   

       application_date     status_cd  
0              20150905  0-00000000-0  
1              20150414  0-00000000-0  
2              20150529  D-20100325-C  
3              20160115  0-00000000-0  
4              20170605  6-20090929-2  
...                 ...           ...  
21966          20171110  0-00000000-0  
21967          20150313  F-20101028-F  
21968          20150406  0-00000000-0  
21969          20160206  0-00000000-0  
21970          20150424  0-00000000-0  

[21971 rows x 11 columns]

In [17]:
def func_post_tokyo(x):
    ret_value = 0
    post3 = x[:3]
    if  "100" <= post3 <="209":
        ret_value = 1
    return ret_value
df53["post_tokyo"] = df53["postal_cd"].apply(func_post_tokyo)
df53

customer_id customer_name  gender_cd gender   birth_day  age  \
0      CS021313000114        大野 あや子          1     女性  1981-04-29   37   
1      CS037613000071         六角 雅彦          9     不明  1952-04-01   66   
2      CS031415000172       宇多田 貴美子          1     女性  1976-10-04   42   
3      CS028811000001        堀井 かおり          1     女性  1933-03-27   86   
4      CS001215000145         田崎 美紀          1     女性  1995-03-29   24   
...               ...           ...        ...    ...         ...  ...   
21966  CS002512000474         市村 夏希          1     女性  1959-10-12   59   
21967  CS029414000065         上村 怜奈          1     女性  1970-10-19   48   
21968  CS012403000043          堀越 明          0     男性  1972-12-16   46   
21969  CS033512000184         池谷 華子          1     女性  1964-06-05   54   
21970  CS009213000022          稲垣 咲          1     女性  1996-08-16   22   

      postal_cd                  address application_store_cd  \
0      259-1113     神奈川県伊勢原市粟窪**********               S14021   
1      136-0076       東京都江東区南砂**********               S13037   
2      151-0053      東京都渋谷区代々木**********               S13031   
3      245-0016   神奈川県横浜市泉区和泉町**********               S14028   
4      144-0055      東京都大田区仲六郷**********               S13001   
...         ...                      ...                  ...   
21966  185-0034      東京都国分寺市光町**********               S13002   
21967  279-0043      千葉県浦安市富士見**********               S12029   
21968  231-0825  神奈川県横浜市中区本牧間門**********               S14012   
21969  245-0016   神奈川県横浜市泉区和泉町**********               S14033   
21970  154-0012      東京都世田谷区駒沢**********               S13009   

       application_date     status_cd  post_tokyo  
0              20150905  0-00000000-0           0  
1              20150414  0-00000000-0           1  
2              20150529  D-20100325-C           1  
3              20160115  0-00000000-0           0  
4              20170605  6-20090929-2           1  
...                 ...           ...         ...  
21966          20171110  0-00000000-0           1  
21967          20150313  F-20101028-F           0  
21968          20150406  0-00000000-0           0  
21969          20160206  0-00000000-0           0  
21970          20150424  0-00000000-0           1  

[21971 rows x 12 columns]

In [64]:
df53_r = df_receipt.copy()
df53_r2 = df53_r.groupby("customer_id")[["amount"]].sum().sort_values("amount").reset_index()
df53_r2

customer_id    amount
0     CS014614000056        70
1     CS002513000267        80
2     CS010803000005        93
3     CS005412000085        98
4     CS035304000008       105
...              ...       ...
8302  CS028415000007     19127
8303  CS031414000051     19202
8304  CS015415000185     20153
8305  CS017415000097     23086
8306  ZZ000000000000  12395003

[8307 rows x 2 columns]

In [65]:
df_tmp = pd.merge(df53[["customer_id","post_tokyo"]], df53_r2["customer_id"], how ='inner', on="customer_id")
df_tmp.groupby("post_tokyo").count()

customer_id
post_tokyo             
0                  3906
1                  4400

# P-54

---
> P-054: 顧客データデータフレーム（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに抽出せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。結果は10件表示させれば良い。

In [66]:
df54 = df_customer.copy()

In [67]:
x = "東京都武蔵野市"
x[:3]

'東京都'

In [71]:
def func_city_cd(x):
    city_str = x[:3]

    ret_value = 0

    if city_str == "埼玉県":
        ret_value = 11
    elif city_str == "千葉県":
        ret_value = 12
    elif city_str == "東京都":
        ret_value = 13
    elif city_str == "神奈川":
        ret_value =14
    
    
    return ret_value
df54["city_cd"] = df54["address"].apply(func_city_cd)
df54[["customer_id","address","city_cd"]].head(10)


customer_id                 address  city_cd
0  CS021313000114    神奈川県伊勢原市粟窪**********       14
1  CS037613000071      東京都江東区南砂**********       13
2  CS031415000172     東京都渋谷区代々木**********       13
3  CS028811000001  神奈川県横浜市泉区和泉町**********       14
4  CS001215000145     東京都大田区仲六郷**********       13
5  CS020401000016      東京都板橋区若木**********       13
6  CS015414000103      東京都江東区北砂**********       13
7  CS029403000008      千葉県浦安市海楽**********       12
8  CS015804000004      東京都江東区北砂**********       13
9  CS033513000180  神奈川県横浜市旭区善部町**********       14

# P-55

---
> P-055: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額と合計ともに表示せよ。カテゴリ値は上から順に1〜4とする。結果は10件表示させれば良い。
>
> - 最小値以上第一四分位未満
> - 第一四分位以上第二四分位未満
> - 第二四分位以上第三四分位未満
> - 第三四分位以上

In [3]:
df55 = df_receipt.copy()
df55_sum = df55.groupby("customer_id")[["amount"]].sum()
quantile_list = list(df55_sum["amount"].quantile([0.0, 0.25, 0.50, 0.75, 1.0]))
quantile_list

[70.0, 548.5, 1478.0, 3651.0, 12395003.0]

In [4]:
df55_sum["quantile_type"] = 4
df55_sum.loc[df55_sum["amount"] < quantile_list[3], "quantile_type"] = 3
df55_sum.loc[df55_sum["amount"] < quantile_list[2], "quantile_type"] = 2
df55_sum.loc[df55_sum["amount"] < quantile_list[1], "quantile_type"] = 1
df55_sum.head(10)

amount  quantile_type
customer_id                          
CS001113000004    1298              2
CS001114000005     626              2
CS001115000010    3044              3
CS001205000004    1988              3
CS001205000006    3337              3
CS001211000025     456              1
CS001212000027     448              1
CS001212000031     296              1
CS001212000046     228              1
CS001212000070     456              1

次回 12/20ここから↓

---
> P-056: 顧客データフレーム（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに抽出せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。先頭10件を表示させればよい。

In [10]:
df56 = df_customer.copy()


In [13]:
def fnc_age(x):
    age = int(x)

    if age >= 60:
        ret = 60
    else:
        ret = age // 10 * 10
    
    return ret
df56["age_area"] = df56["age"].apply(fnc_age)
df56[["customer_id","birth_day","age_area"]].head(10)


customer_id   birth_day  age_area
0  CS021313000114  1981-04-29        30
1  CS037613000071  1952-04-01        60
2  CS031415000172  1976-10-04        40
3  CS028811000001  1933-03-27        60
4  CS001215000145  1995-03-29        20
5  CS020401000016  1974-09-15        40
6  CS015414000103  1977-08-09        40
7  CS029403000008  1973-08-17        40
8  CS015804000004  1931-05-02        60
9  CS033513000180  1962-07-11        50

---
> P-057: 前問題の抽出結果と性別（gender）を組み合わせ、新たに性別×年代の組み合わせを表すカテゴリデータを作成せよ。組み合わせを表すカテゴリの値は任意とする。先頭10件を表示させればよい。

In [14]:
df56

customer_id customer_name  gender_cd gender   birth_day  age  \
0      CS021313000114        大野 あや子          1     女性  1981-04-29   37   
1      CS037613000071         六角 雅彦          9     不明  1952-04-01   66   
2      CS031415000172       宇多田 貴美子          1     女性  1976-10-04   42   
3      CS028811000001        堀井 かおり          1     女性  1933-03-27   86   
4      CS001215000145         田崎 美紀          1     女性  1995-03-29   24   
...               ...           ...        ...    ...         ...  ...   
21966  CS002512000474         市村 夏希          1     女性  1959-10-12   59   
21967  CS029414000065         上村 怜奈          1     女性  1970-10-19   48   
21968  CS012403000043          堀越 明          0     男性  1972-12-16   46   
21969  CS033512000184         池谷 華子          1     女性  1964-06-05   54   
21970  CS009213000022          稲垣 咲          1     女性  1996-08-16   22   

      postal_cd                  address application_store_cd  \
0      259-1113     神奈川県伊勢原市粟窪**********               S14021   
1      136-0076       東京都江東区南砂**********               S13037   
2      151-0053      東京都渋谷区代々木**********               S13031   
3      245-0016   神奈川県横浜市泉区和泉町**********               S14028   
4      144-0055      東京都大田区仲六郷**********               S13001   
...         ...                      ...                  ...   
21966  185-0034      東京都国分寺市光町**********               S13002   
21967  279-0043      千葉県浦安市富士見**********               S12029   
21968  231-0825  神奈川県横浜市中区本牧間門**********               S14012   
21969  245-0016   神奈川県横浜市泉区和泉町**********               S14033   
21970  154-0012      東京都世田谷区駒沢**********               S13009   

       application_date     status_cd  age_area  
0              20150905  0-00000000-0        30  
1              20150414  0-00000000-0        60  
2              20150529  D-20100325-C        40  
3              20160115  0-00000000-0        60  
4              20170605  6-20090929-2        20  
...                 ...           ...       ...  
21966          20171110  0-00000000-0        50  
21967          20150313  F-20101028-F        40  
21968          20150406  0-00000000-0        40  
21969          20160206  0-00000000-0        50  
21970          20150424  0-00000000-0        20  

[21971 rows x 12 columns]

In [17]:
df56["gender_age"] = df56["gender"].astype("str") + ":" + df56["age_area"].astype("str") + "代"
df56[["customer_id","birth_day","age","age_area","gender_age"]].head(10)

customer_id   birth_day  age  age_area gender_age
0  CS021313000114  1981-04-29   37        30     女性:30代
1  CS037613000071  1952-04-01   66        60     不明:60代
2  CS031415000172  1976-10-04   42        40     女性:40代
3  CS028811000001  1933-03-27   86        60     女性:60代
4  CS001215000145  1995-03-29   24        20     女性:20代
5  CS020401000016  1974-09-15   44        40     男性:40代
6  CS015414000103  1977-08-09   41        40     女性:40代
7  CS029403000008  1973-08-17   45        40     男性:40代
8  CS015804000004  1931-05-02   87        60     男性:60代
9  CS033513000180  1962-07-11   56        50     女性:50代

---
> P-058: 顧客データフレーム（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに抽出せよ。結果は10件表示させれば良い。

In [20]:
#kameちゃん先生とゆかりお先生の方が、かっこいい解き方
df58 = df_customer.copy()
df58_2 = pd.get_dummies(df58.gender_cd)
df58_3 = pd.concat([df58["customer_id"], df58_2], axis=1)
df58_3.head(10)

customer_id  0  1  9
0  CS021313000114  0  1  0
1  CS037613000071  0  0  1
2  CS031415000172  0  1  0
3  CS028811000001  0  1  0
4  CS001215000145  0  1  0
5  CS020401000016  1  0  0
6  CS015414000103  0  1  0
7  CS029403000008  1  0  0
8  CS015804000004  1  0  0
9  CS033513000180  0  1  0

---
> P-059: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに表示せよ。標準化に使用する標準偏差は、不偏標準偏差と標本標準偏差のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [21]:
df59 = df_receipt.copy()
df59_amount = df59[(~df59["customer_id"].str.startswith("Z"))].groupby("customer_id")[["amount"]].sum()
df59_amount

amount
customer_id           
CS001113000004    1298
CS001114000005     626
CS001115000010    3044
CS001205000004    1988
CS001205000006    3337
...                ...
CS051212000001     336
CS051513000004     551
CS051515000002     265
CS052212000002     192
CS052514000001     178

[8306 rows x 1 columns]

---
> P-060: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [22]:
df60 = df_receipt.copy()
df60_amount = df59[(~df59["customer_id"].str.startswith("Z"))].groupby("customer_id")[["amount"]].sum()
df60_amount

amount
customer_id           
CS001113000004    1298
CS001114000005     626
CS001115000010    3044
CS001205000004    1988
CS001205000006    3337
...                ...
CS051212000001     336
CS051513000004     551
CS051515000002     265
CS052212000002     192
CS052514000001     178

[8306 rows x 1 columns]

---
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [30]:
df61 = df_receipt.copy()
df61_amount = df59[(~df59["customer_id"].str.startswith("Z"))].groupby("customer_id")[["amount"]].sum()
df61_amount["log10"] = df61_amount["amount"].apply((lambda x: math.log10(x)))
df61_amount.head(10)

amount     log10
customer_id                     
CS001113000004    1298  3.113275
CS001114000005     626  2.796574
CS001115000010    3044  3.483445
CS001205000004    1988  3.298416
CS001205000006    3337  3.523356
CS001211000025     456  2.658965
CS001212000027     448  2.651278
CS001212000031     296  2.471292
CS001212000046     228  2.357935
CS001212000070     456  2.658965

---
> P-062: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を自然対数化(底=e）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [3]:
df62 = df_receipt.copy()
df62 = df62[~df_receipt["customer_id"].str.startswith("Z")]
df62_sum = df62.groupby("customer_id")[["amount"]].sum().reset_index()
df62_sum["log"]= np.log(df62_sum["amount"])
df62_sum.head(10)

customer_id  amount       log
0  CS001113000004    1298  7.168580
1  CS001114000005     626  6.439350
2  CS001115000010    3044  8.020928
3  CS001205000004    1988  7.594884
4  CS001205000006    3337  8.112827
5  CS001211000025     456  6.122493
6  CS001212000027     448  6.104793
7  CS001212000031     296  5.690359
8  CS001212000046     228  5.429346
9  CS001212000070     456  6.122493

---
> P-063: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益額を算出せよ。結果は10件表示させれば良い。

In [5]:
df63 = df_product.copy()
df63.head()
df63["unit_profit"] = df63["unit_price"] - df63["unit_cost"]
df63.head(10)

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040101001                  4                 401              40101   
1  P040101002                  4                 401              40101   
2  P040101003                  4                 401              40101   
3  P040101004                  4                 401              40101   
4  P040101005                  4                 401              40101   
5  P040101006                  4                 401              40101   
6  P040101007                  4                 401              40101   
7  P040101008                  4                 401              40101   
8  P040101009                  4                 401              40101   
9  P040101010                  4                 401              40101   

   unit_price  unit_cost  unit_profit  
0       198.0      149.0         49.0  
1       218.0      164.0         54.0  
2       230.0      173.0         57.0  
3       248.0      186.0         62.0  
4       268.0      201.0         67.0  
5       298.0      224.0         74.0  
6       338.0      254.0         84.0  
7       420.0      315.0        105.0  
8       498.0      374.0        124.0  
9       580.0      435.0        145.0

---
> P-064: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。
ただし、単価と原価にはNULLが存在することに注意せよ。

In [6]:
df64 = df_product.copy()
df64["unit_profit_rate"] = (df64["unit_price"] - df64["unit_cost"])/df64["unit_price"]
df64["unit_profit_rate"].mean(skipna=True)

0.24911389885177

---
> P-065: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [7]:
df65 = df_product.copy()
df65["new_unit_price"] = df65["unit_cost"]/0.7
df65["new_unit_price"] = df65["new_unit_price"].fillna(0).astype(int, errors='ignore')
df65.head(10)

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040101001                  4                 401              40101   
1  P040101002                  4                 401              40101   
2  P040101003                  4                 401              40101   
3  P040101004                  4                 401              40101   
4  P040101005                  4                 401              40101   
5  P040101006                  4                 401              40101   
6  P040101007                  4                 401              40101   
7  P040101008                  4                 401              40101   
8  P040101009                  4                 401              40101   
9  P040101010                  4                 401              40101   

   unit_price  unit_cost  new_unit_price  
0       198.0      149.0             212  
1       218.0      164.0             234  
2       230.0      173.0             247  
3       248.0      186.0             265  
4       268.0      201.0             287  
5       298.0      224.0             320  
6       338.0      254.0             362  
7       420.0      315.0             450  
8       498.0      374.0             534  
9       580.0      435.0             621

In [8]:
df65["unit_profit_rate"] = (df65["new_unit_price"] - df65["unit_cost"])/df65["new_unit_price"]
df65["unit_profit_rate"].mean(skipna=True)

0.29867844972541435

---
> P-066: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を四捨五入すること（0.5については偶数方向の丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [9]:
df66 = df_product.copy()
df66["new_unit_price"] = df66["unit_cost"]/0.7
df66["new_unit_price"] = df66["new_unit_price"].fillna(0).round().astype(int, errors='ignore')
df66.head(10)

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040101001                  4                 401              40101   
1  P040101002                  4                 401              40101   
2  P040101003                  4                 401              40101   
3  P040101004                  4                 401              40101   
4  P040101005                  4                 401              40101   
5  P040101006                  4                 401              40101   
6  P040101007                  4                 401              40101   
7  P040101008                  4                 401              40101   
8  P040101009                  4                 401              40101   
9  P040101010                  4                 401              40101   

   unit_price  unit_cost  new_unit_price  
0       198.0      149.0             213  
1       218.0      164.0             234  
2       230.0      173.0             247  
3       248.0      186.0             266  
4       268.0      201.0             287  
5       298.0      224.0             320  
6       338.0      254.0             363  
7       420.0      315.0             450  
8       498.0      374.0             534  
9       580.0      435.0             621

---
> P-067: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [11]:
import math
df67 = df_product.copy()
df67["new_unit_price"] = df67["unit_cost"]/0.7
df67["new_unit_price"] = df67["new_unit_price"].fillna(0).apply(math.ceil)
df67.head(10)

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040101001                  4                 401              40101   
1  P040101002                  4                 401              40101   
2  P040101003                  4                 401              40101   
3  P040101004                  4                 401              40101   
4  P040101005                  4                 401              40101   
5  P040101006                  4                 401              40101   
6  P040101007                  4                 401              40101   
7  P040101008                  4                 401              40101   
8  P040101009                  4                 401              40101   
9  P040101010                  4                 401              40101   

   unit_price  unit_cost  new_unit_price  
0       198.0      149.0             213  
1       218.0      164.0             235  
2       230.0      173.0             248  
3       248.0      186.0             266  
4       268.0      201.0             288  
5       298.0      224.0             320  
6       338.0      254.0             363  
7       420.0      315.0             451  
8       498.0      374.0             535  
9       580.0      435.0             622

In [7]:
df67["riekiritsu"] = (df67["new_unit_price"] - df67["unit_cost"])/df67["new_unit_price"]
df67

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0      P040101001                  4                 401              40101   
1      P040101002                  4                 401              40101   
2      P040101003                  4                 401              40101   
3      P040101004                  4                 401              40101   
4      P040101005                  4                 401              40101   
...           ...                ...                 ...                ...   
10025  P091503001                  9                 915              91503   
10026  P091503002                  9                 915              91503   
10027  P091503003                  9                 915              91503   
10028  P091503004                  9                 915              91503   
10029  P091503005                  9                 915              91503   

       unit_price  unit_cost  new_unit_price  riekiritsu  
0           198.0      149.0             213    0.300469  
1           218.0      164.0             235    0.302128  
2           230.0      173.0             248    0.302419  
3           248.0      186.0             266    0.300752  
4           268.0      201.0             288    0.302083  
...           ...        ...             ...         ...  
10025       280.0      210.0             300    0.300000  
10026       680.0      510.0             729    0.300412  
10027      1080.0      810.0            1158    0.300518  
10028      1130.0      848.0            1212    0.300330  
10029      1280.0      960.0            1372    0.300292  

[10030 rows x 8 columns]

---
> P-068: 商品データフレーム（df_product）の各商品について、消費税率10%の税込み金額を求めよ。 1円未満の端数は切り捨てとし、結果は10件表示すれば良い。ただし、単価（unit_price）にはNULLが存在することに注意せよ。

In [13]:
df68 = df_product.copy()
df68["tax_in_price"] = df68["unit_price"] * 1.10
df68["tax_in_price"] = df68["tax_in_price"].fillna(0).apply(np.floor)
df68.head(10)

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040101001                  4                 401              40101   
1  P040101002                  4                 401              40101   
2  P040101003                  4                 401              40101   
3  P040101004                  4                 401              40101   
4  P040101005                  4                 401              40101   
5  P040101006                  4                 401              40101   
6  P040101007                  4                 401              40101   
7  P040101008                  4                 401              40101   
8  P040101009                  4                 401              40101   
9  P040101010                  4                 401              40101   

   unit_price  unit_cost  tax_in_price  
0       198.0      149.0         217.0  
1       218.0      164.0         239.0  
2       230.0      173.0         253.0  
3       248.0      186.0         272.0  
4       268.0      201.0         294.0  
5       298.0      224.0         327.0  
6       338.0      254.0         371.0  
7       420.0      315.0         462.0  
8       498.0      374.0         547.0  
9       580.0      435.0         638.0

---
> P-069: レシート明細データフレーム（df_receipt）と商品データフレーム（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分"07"（瓶詰缶詰）の購入実績がある顧客のみとし、結果は10件表示させればよい。

In [15]:
df69_r = df_receipt.copy()
df69_r

sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
0        20181103   1541203200   S14006         112               1   
1        20181118   1542499200   S13008        1132               2   
2        20170712   1499817600   S14028        1102               1   
3        20190205   1549324800   S14042        1132               1   
4        20180821   1534809600   S14025        1102               2   
...           ...          ...      ...         ...             ...   
104676   20180221   1519171200   S13043        1132               2   
104677   20190911   1568160000   S14047        1132               2   
104678   20170311   1489190400   S14040        1122               1   
104679   20170331   1490918400   S13002        1142               1   
104680   20190423   1555977600   S13016        1102               2   

           customer_id  product_cd  quantity  amount  
0       CS006214000001  P070305012         1     158  
1       CS008415000097  P070701017         1      81  
2       CS028414000014  P060101005         1     170  
3       ZZ000000000000  P050301001         1      25  
4       CS025415000050  P060102007         1      90  
...                ...         ...       ...     ...  
104676  ZZ000000000000  P050101001         1      40  
104677  ZZ000000000000  P071006005         1     218  
104678  CS040513000195  P050405003         1     168  
104679  CS002513000049  P060303001         1     148  
104680  ZZ000000000000  P050601001         1     138  

[104681 rows x 9 columns]

In [16]:
df69_p = df_product.copy()
df69_p_cat7 = df69_p[df69_p["category_major_cd"] == 7]
df69_p_cat7

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
1609  P070101001                  7                 701              70101   
1610  P070101002                  7                 701              70101   
1611  P070101003                  7                 701              70101   
1612  P070101004                  7                 701              70101   
1613  P070101005                  7                 701              70101   
...          ...                ...                 ...                ...   
5669  P071401021                  7                 714              71401   
5670  P071401022                  7                 714              71401   
5671  P071401023                  7                 714              71401   
5672  P071401024                  7                 714              71401   
5673  P071401025                  7                 714              71401   

      unit_price  unit_cost  
1609        38.0       29.0  
1610        45.0       34.0  
1611        48.0       36.0  
1612        55.0       41.0  
1613        70.0       53.0  
...          ...        ...  
5669      2400.0     1800.0  
5670      2400.0     1800.0  
5671      2400.0     1800.0  
5672      2400.0     1800.0  
5673      2400.0     1800.0  

[4065 rows x 6 columns]

In [17]:
df_69_m1 = pd.merge(df69_r[["customer_id","product_cd","amount"]], df69_p[["product_cd","category_major_cd"]], how ='inner', on="product_cd")
df_69_sum1 = df_69_m1.groupby("customer_id")[["amount"]].sum()
df_69_sum1

amount
customer_id             
CS001113000004      1298
CS001114000005       626
CS001115000010      3044
CS001205000004      1988
CS001205000006      3337
...                  ...
CS051513000004       551
CS051515000002       265
CS052212000002       192
CS052514000001       178
ZZ000000000000  12395003

[8307 rows x 1 columns]

In [18]:
df_69_m2 = pd.merge(df69_r[["customer_id","product_cd","amount"]], df69_p_cat7[["product_cd","category_major_cd"]], how ='inner', on="product_cd")
df_69_sum2 = df_69_m2.groupby("customer_id")[["amount"]].sum()
df_69_sum2

amount
customer_id            
CS001113000004     1298
CS001114000005      486
CS001115000010     2694
CS001205000004      346
CS001205000006     2004
...                 ...
CS051212000001       68
CS051513000004      233
CS052212000002      102
CS052514000001       68
ZZ000000000000  6943009

[6865 rows x 1 columns]

In [21]:
df_69_sum1 = df_69_sum1.rename(columns={"amount":"all_amount"})

In [22]:
df69_m3 = pd.merge(df_69_sum1, df_69_sum2, how ='inner', on="customer_id")
df69_m3.head(10)

all_amount  amount
customer_id                       
CS001113000004        1298    1298
CS001114000005         626     486
CS001115000010        3044    2694
CS001205000004        1988     346
CS001205000006        3337    2004
CS001212000027         448     200
CS001212000031         296     296
CS001212000046         228     108
CS001212000070         456     308
CS001213000018         243     145

In [23]:
df69_m3["hiritu"] = df69_m3["amount"] / df69_m3["all_amount"] * 100
df69_m3.head(10)

all_amount  amount      hiritu
customer_id                                   
CS001113000004        1298    1298  100.000000
CS001114000005         626     486   77.635783
CS001115000010        3044    2694   88.501971
CS001205000004        1988     346   17.404427
CS001205000006        3337    2004   60.053941
CS001212000027         448     200   44.642857
CS001212000031         296     296  100.000000
CS001212000046         228     108   47.368421
CS001212000070         456     308   67.543860
CS001213000018         243     145   59.670782

---
> P-070: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [6]:
df70_r = df_receipt.copy()
df70_r

sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
0        20181103   1541203200   S14006         112               1   
1        20181118   1542499200   S13008        1132               2   
2        20170712   1499817600   S14028        1102               1   
3        20190205   1549324800   S14042        1132               1   
4        20180821   1534809600   S14025        1102               2   
...           ...          ...      ...         ...             ...   
104676   20180221   1519171200   S13043        1132               2   
104677   20190911   1568160000   S14047        1132               2   
104678   20170311   1489190400   S14040        1122               1   
104679   20170331   1490918400   S13002        1142               1   
104680   20190423   1555977600   S13016        1102               2   

           customer_id  product_cd  quantity  amount  
0       CS006214000001  P070305012         1     158  
1       CS008415000097  P070701017         1      81  
2       CS028414000014  P060101005         1     170  
3       ZZ000000000000  P050301001         1      25  
4       CS025415000050  P060102007         1      90  
...                ...         ...       ...     ...  
104676  ZZ000000000000  P050101001         1      40  
104677  ZZ000000000000  P071006005         1     218  
104678  CS040513000195  P050405003         1     168  
104679  CS002513000049  P060303001         1     148  
104680  ZZ000000000000  P050601001         1     138  

[104681 rows x 9 columns]

In [7]:
df70_c = df_customer.copy()
df70_c

customer_id customer_name  gender_cd gender   birth_day  age  \
0      CS021313000114        大野 あや子          1     女性  1981-04-29   37   
1      CS037613000071         六角 雅彦          9     不明  1952-04-01   66   
2      CS031415000172       宇多田 貴美子          1     女性  1976-10-04   42   
3      CS028811000001        堀井 かおり          1     女性  1933-03-27   86   
4      CS001215000145         田崎 美紀          1     女性  1995-03-29   24   
...               ...           ...        ...    ...         ...  ...   
21966  CS002512000474         市村 夏希          1     女性  1959-10-12   59   
21967  CS029414000065         上村 怜奈          1     女性  1970-10-19   48   
21968  CS012403000043          堀越 明          0     男性  1972-12-16   46   
21969  CS033512000184         池谷 華子          1     女性  1964-06-05   54   
21970  CS009213000022          稲垣 咲          1     女性  1996-08-16   22   

      postal_cd                  address application_store_cd  \
0      259-1113     神奈川県伊勢原市粟窪**********               S14021   
1      136-0076       東京都江東区南砂**********               S13037   
2      151-0053      東京都渋谷区代々木**********               S13031   
3      245-0016   神奈川県横浜市泉区和泉町**********               S14028   
4      144-0055      東京都大田区仲六郷**********               S13001   
...         ...                      ...                  ...   
21966  185-0034      東京都国分寺市光町**********               S13002   
21967  279-0043      千葉県浦安市富士見**********               S12029   
21968  231-0825  神奈川県横浜市中区本牧間門**********               S14012   
21969  245-0016   神奈川県横浜市泉区和泉町**********               S14033   
21970  154-0012      東京都世田谷区駒沢**********               S13009   

       application_date     status_cd  
0              20150905  0-00000000-0  
1              20150414  0-00000000-0  
2              20150529  D-20100325-C  
3              20160115  0-00000000-0  
4              20170605  6-20090929-2  
...                 ...           ...  
21966          20171110  0-00000000-0  
21967          20150313  F-20101028-F  
21968          20150406  0-00000000-0  
21969          20160206  0-00000000-0  
21970          20150424  0-00000000-0  

[21971 rows x 11 columns]

In [8]:
df70_m1 = pd.merge(df70_r[["customer_id","sales_ymd"]], df70_c[["customer_id","application_date"]], how ='inner', on="customer_id")
df70_m1

customer_id  sales_ymd  application_date
0      CS006214000001   20181103          20150201
1      CS006214000001   20170509          20150201
2      CS006214000001   20170608          20150201
3      CS006214000001   20170608          20150201
4      CS006214000001   20181028          20150201
...               ...        ...               ...
65677  CS004613000146   20171202          20160813
65678  CS002314000037   20180421          20151202
65679  CS002314000037   20180421          20151202
65680  CS040311000022   20190416          20150419
65681  CS040311000022   20190416          20150419

[65682 rows x 3 columns]

In [9]:
df70_m1["application_date2"] = pd.to_datetime(df70_m1["application_date"],format="%Y%m%d")
df70_m1["sales_ymd2"] = pd.to_datetime(df70_m1["sales_ymd"].astype(str),format="%Y%m%d")
df70_m1["days_after_applied"] = df70_m1["sales_ymd2"] - df70_m1["application_date2"]
df70_m1[["customer_id","sales_ymd","application_date","days_after_applied"]].head(10)

customer_id  sales_ymd  application_date days_after_applied
0  CS006214000001   20181103          20150201          1371 days
1  CS006214000001   20170509          20150201           828 days
2  CS006214000001   20170608          20150201           858 days
3  CS006214000001   20170608          20150201           858 days
4  CS006214000001   20181028          20150201          1365 days
5  CS006214000001   20181028          20150201          1365 days
6  CS006214000001   20170509          20150201           828 days
7  CS006214000001   20190908          20150201          1680 days
8  CS006214000001   20180131          20150201          1095 days
9  CS006214000001   20170705          20150201           885 days

---
> P-071: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [10]:
df71 = df70_m1.copy()
df71["days_after_applied"] = df71["days_after_applied"].apply(lambda x: x if x > pd.Timedelta('30 days') else 0)
df71.head(10)

customer_id  sales_ymd  application_date application_date2 sales_ymd2  \
0  CS006214000001   20181103          20150201        2015-02-01 2018-11-03   
1  CS006214000001   20170509          20150201        2015-02-01 2017-05-09   
2  CS006214000001   20170608          20150201        2015-02-01 2017-06-08   
3  CS006214000001   20170608          20150201        2015-02-01 2017-06-08   
4  CS006214000001   20181028          20150201        2015-02-01 2018-10-28   
5  CS006214000001   20181028          20150201        2015-02-01 2018-10-28   
6  CS006214000001   20170509          20150201        2015-02-01 2017-05-09   
7  CS006214000001   20190908          20150201        2015-02-01 2019-09-08   
8  CS006214000001   20180131          20150201        2015-02-01 2018-01-31   
9  CS006214000001   20170705          20150201        2015-02-01 2017-07-05   

  days_after_applied  
0          1371 days  
1           828 days  
2           858 days  
3           858 days  
4          1365 days  
5          1365 days  
6           828 days  
7          1680 days  
8          1095 days  
9           885 days

---
> P-072: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い。（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [11]:
df72 = pd.merge(df_receipt,df_customer)[["customer_id", "sales_ymd", "application_date"]]
df72["x"] = pd.to_datetime(df72["sales_ymd"].astype(str))
df72["y"] = pd.to_datetime(df72["application_date"].astype(str))
df72["days"] = df72["x"] - df72["y"]
df72["years"] = df72["days"] // np.timedelta64(1, 'Y')
df72 = df72[["customer_id", "sales_ymd", "application_date", "years"]]
df72.head(10)

customer_id  sales_ymd  application_date  years
0  CS006214000001   20181103          20150201      3
1  CS006214000001   20170509          20150201      2
2  CS006214000001   20170608          20150201      2
3  CS006214000001   20170608          20150201      2
4  CS006214000001   20181028          20150201      3
5  CS006214000001   20181028          20150201      3
6  CS006214000001   20170509          20150201      2
7  CS006214000001   20190908          20150201      4
8  CS006214000001   20180131          20150201      2
9  CS006214000001   20170705          20150201      2

---
> P-073: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [12]:
df73 = pd.merge(df_receipt,df_customer)[["customer_id", "sales_ymd", "application_date"]]
df73["x"] = pd.to_datetime(df73["sales_ymd"].astype(str))
df73["y"] = pd.to_datetime(df73["application_date"].astype(str))
df73["days"] = df73["x"] - df73["y"]
df73["milliseconds"] = df73["days"] // pd.Timedelta("1ms")
df73 = df73[["customer_id", "sales_ymd", "application_date", "milliseconds"]]
df73.head(10)

customer_id  sales_ymd  application_date  milliseconds
0  CS006214000001   20181103          20150201  118454400000
1  CS006214000001   20170509          20150201   71539200000
2  CS006214000001   20170608          20150201   74131200000
3  CS006214000001   20170608          20150201   74131200000
4  CS006214000001   20181028          20150201  117936000000
5  CS006214000001   20181028          20150201  117936000000
6  CS006214000001   20170509          20150201   71539200000
7  CS006214000001   20190908          20150201  145152000000
8  CS006214000001   20180131          20150201   94608000000
9  CS006214000001   20170705          20150201   76464000000

---
> P-074: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、当該週の月曜日付とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値でデータを保持している点に注意）。

In [14]:
from datetime import timedelta
def get_monday(x):
    w = x.weekday()
    return x - timedelta(days=w)

In [15]:
df74 = df_receipt.copy()
df74["sales_date"] = pd.to_datetime(df74["sales_ymd"].astype(str))
df74["monday"] = df74["sales_date"].apply(get_monday)
df74["w"] = df74["sales_date"].dt.weekday
df74 = df74[["sales_ymd","w", "monday"]]
df74.head()

sales_ymd  w     monday
0   20181103  5 2018-10-29
1   20181118  6 2018-11-12
2   20170712  2 2017-07-10
3   20190205  1 2019-02-04
4   20180821  1 2018-08-20

---
> P-075: 顧客データフレーム（df_customer）からランダムに1%のデータを抽出し、先頭から10件データを抽出せよ。

In [16]:
# https://note.nkmk.me/python-pandas-sample/
df75 = df_receipt.copy()
df75.sample(frac=0.01).head(10)

sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
54680   20170810   1502323200   S13004        1102               1   
10421   20190129   1548720000   S13038        1152               1   
3239    20170417   1492387200   S13017        1112               2   
92539   20170213   1486944000   S13018        1172               1   
1587    20170208   1486512000   S14034        1142               2   
5454    20170802   1501632000   S13009        1102               2   
96123   20180906   1536192000   S13002        1162               1   
7255    20170822   1503360000   S14012        1122               1   
29120   20181203   1543795200   S14033        1182               1   
27314   20170528   1495929600   S14028        1102               2   

          customer_id  product_cd  quantity  amount  
54680  CS004414000043  P090302016         1     120  
10421  CS038414000063  P080702001         1     110  
3239   CS017815000001  P050101005         1     145  
92539  CS018415000131  P060602003         1      78  
1587   ZZ000000000000  P071003046         1     398  
5454   CS009414000082  P071005019         1     102  
96123  CS002515000032  P070401080         1     298  
7255   CS012414000027  P070602035         1     168  
29120  CS033514000006  P071102005         1     698  
27314  CS028415000086  P040101001         1     198

---
> P-076: 顧客データフレーム（df_customer）から性別（gender_cd）の割合に基づきランダムに10%のデータを層化抽出データし、性別ごとに件数を集計せよ。

In [20]:
df_076 = df_customer.copy()
df_076.groupby("gender_cd").sample(frac=0.1)["gender_cd"].value_counts()

1    1792
0     298
9     107
Name: gender_cd, dtype: int64

---
> P-077: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を平均から3σ以上離れたものとする。結果は10件表示させれば良い。

In [21]:
df77 = df_receipt.copy()
df77 = df77[~df77["customer_id"].str.startswith("Z")]
df77_sum = df77.groupby("customer_id")[["amount"]].sum()

df77_average = np.mean(df77_sum["amount"])
df77_sd = np.std(df77_sum["amount"])
# 外れ値の基準点
outlier_min = df77_average - (df77_sd) * 3
outlier_max = df77_average + (df77_sd) * 3
df77_sum[(df77_sum["amount"] <= outlier_min) | (df77_sum["amount"] >= outlier_max)].head(10)

amount
customer_id           
CS001605000009   18925
CS006415000147   12723
CS006515000023   18372
CS006515000125   12575
CS006515000209   11373
CS007115000006   11528
CS007514000056   13293
CS007514000094   15735
CS007515000107   11188
CS007615000026   11959

---
> P-078: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第一四分位と第三四分位の差であるIQRを用いて、「第一四分位数-1.5×IQR」よりも下回るもの、または「第三四分位数+1.5×IQR」を超えるものとする。結果は10件表示させれば良い。

In [2]:
#https://life-freedom888.com/pandas-yonbuni/
df78 = df_receipt.copy()
df78 = df78[~df78["customer_id"].str.startswith("Z")]
df78_sum = df78.groupby("customer_id")[["amount"]].sum()
df78_sum

amount
customer_id           
CS001113000004    1298
CS001114000005     626
CS001115000010    3044
CS001205000004    1988
CS001205000006    3337
...                ...
CS051212000001     336
CS051513000004     551
CS051515000002     265
CS052212000002     192
CS052514000001     178

[8306 rows x 1 columns]

In [3]:
df78_quantile1 = df78["amount"].quantile(0.25)
df78_quantile3 = df78["amount"].quantile(0.75)
df78_IRQ = df78_quantile3 - df78_quantile1
df78_IRQ


186.0

In [4]:
# 外れ値の基準点
outlier_min = df78_quantile1 - 1.5 * df78_IRQ
outlier_max = df78_quantile3 + 1.5 * df78_IRQ
df78_sum[(df78_sum["amount"] < outlier_min) | (df78_sum["amount"] > outlier_max)].head(10)

amount
customer_id           
CS001113000004    1298
CS001114000005     626
CS001115000010    3044
CS001205000004    1988
CS001205000006    3337
CS001214000009    4685
CS001214000017    4132
CS001214000048    2374
CS001214000052    5639
CS001215000005     735

---
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

In [8]:
#https://note.nkmk.me/python-pandas-nan-judge-count/
df79 = df_product.copy()
#isnull で欠損値(NaN)かどうか確認→sum でTrueの数を合計
df79.isnull().sum()
#df79.isnull().value_counts()


product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64

---
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [10]:
df80 = df_product.copy()
df80.count()

product_cd            10030
category_major_cd     10030
category_medium_cd    10030
category_small_cd     10030
unit_price            10023
unit_cost             10023
dtype: int64

In [12]:
#https://qiita.com/yuta-38/items/122e607770b88d445d2e#%EF%BC%97nan%E3%82%92%E5%89%8A%E9%99%A4%E3%81%97%E3%81%9F%E3%81%84%E8%A1%8C%E5%88%97%E3%82%92%E6%8C%87%E5%AE%9A
#dropnaよく使う
#df_product_1 = df80.dropna(subset=["product_cd","category_major_cd","category_medium_cd","category_small_cd","unit_price","unit_cost"])
#subsetなくてもOK
df_product_1 = df80.dropna()
df_product_1

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0      P040101001                  4                 401              40101   
1      P040101002                  4                 401              40101   
2      P040101003                  4                 401              40101   
3      P040101004                  4                 401              40101   
4      P040101005                  4                 401              40101   
...           ...                ...                 ...                ...   
10025  P091503001                  9                 915              91503   
10026  P091503002                  9                 915              91503   
10027  P091503003                  9                 915              91503   
10028  P091503004                  9                 915              91503   
10029  P091503005                  9                 915              91503   

       unit_price  unit_cost  
0           198.0      149.0  
1           218.0      164.0  
2           230.0      173.0  
3           248.0      186.0  
4           268.0      201.0  
...           ...        ...  
10025       280.0      210.0  
10026       680.0      510.0  
10027      1080.0      810.0  
10028      1130.0      848.0  
10029      1280.0      960.0  

[10023 rows x 6 columns]

In [59]:
df_product_1.count()

product_cd            10023
category_major_cd     10023
category_medium_cd    10023
category_small_cd     10023
unit_price            10023
unit_cost             10023
dtype: int64

---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [17]:
df81 = df_product.copy()
df_product_nan = df81[df81.isna().any(axis=1)]
df_product_nan

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
159   P040802007                  4                 408              40802   
196   P050103021                  5                 501              50103   
496   P050405009                  5                 504              50405   
1531  P060802026                  6                 608              60802   
2012  P070202092                  7                 702              70202   
6296  P080504027                  8                 805              80504   
7075  P090204185                  9                 902              90204   

      unit_price  unit_cost  
159          NaN        NaN  
196          NaN        NaN  
496          NaN        NaN  
1531         NaN        NaN  
2012         NaN        NaN  
6296         NaN        NaN  
7075         NaN        NaN

In [19]:
df_product_2 = df_product.copy()

In [20]:
#欠損値うめる
#https://pyhoo.jp/dropna-fillna#%E6%AC%A0%E6%90%8D%E5%80%A4%E3%82%92%E5%88%97%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AE%E2%80%9D%E5%B9%B3%E5%9D%87%E5%80%A4%E2%80%9D%E3%81%A7%E7%A9%B4%E5%9F%8B%E3%82%81
#偶数丸め
#https://note.nkmk.me/python-pandas-round-decimal/
df_product_2["unit_price"]=df_product_2["unit_price"].fillna(df_product_2["unit_price"].mean(skipna=True)).round()
df_product_2["unit_cost"]=df_product_2["unit_cost"].fillna(df_product_2["unit_cost"].mean(skipna=True)).round()


In [6]:
pd.merge(df_product_2, df_product_nan, on = "product_cd", suffixes=("_new", "_NaN"))

product_cd  category_major_cd_new  category_medium_cd_new  \
0  P040802007                      4                     408   
1  P050103021                      5                     501   
2  P050405009                      5                     504   
3  P060802026                      6                     608   
4  P070202092                      7                     702   
5  P080504027                      8                     805   
6  P090204185                      9                     902   

   category_small_cd_new  unit_price_new  unit_cost_new  \
0                  40802           403.0          302.0   
1                  50103           403.0          302.0   
2                  50405           403.0          302.0   
3                  60802           403.0          302.0   
4                  70202           403.0          302.0   
5                  80504           403.0          302.0   
6                  90204           403.0          302.0   

   category_major_cd_NaN  category_medium_cd_NaN  category_small_cd_NaN  \
0                      4                     408                  40802   
1                      5                     501                  50103   
2                      5                     504                  50405   
3                      6                     608                  60802   
4                      7                     702                  70202   
5                      8                     805                  80504   
6                      9                     902                  90204   

   unit_price_NaN  unit_cost_NaN  
0             NaN            NaN  
1             NaN            NaN  
2             NaN            NaN  
3             NaN            NaN  
4             NaN            NaN  
5             NaN            NaN  
6             NaN            NaN

In [21]:
df_product_2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10030 entries, 0 to 10029
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   product_cd          10030 non-null  object 
 1   category_major_cd   10030 non-null  int64  
 2   category_medium_cd  10030 non-null  int64  
 3   category_small_cd   10030 non-null  int64  
 4   unit_price          10030 non-null  float64
 5   unit_cost           10030 non-null  float64
dtypes: float64(2), int64(3), object(1)
memory usage: 470.3+ KB


In [22]:
df81 = df_product.copy()
df81["unit_price"].mean(skipna=True).round()

403.0

In [23]:
pd.__version__


'1.1.3'

In [7]:
df_product_2.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たなdf_product_3を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [8]:
df_product_nan

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
159   P040802007                  4                 408              40802   
196   P050103021                  5                 501              50103   
496   P050405009                  5                 504              50405   
1531  P060802026                  6                 608              60802   
2012  P070202092                  7                 702              70202   
6296  P080504027                  8                 805              80504   
7075  P090204185                  9                 902              90204   

      unit_price  unit_cost  
159          NaN        NaN  
196          NaN        NaN  
496          NaN        NaN  
1531         NaN        NaN  
2012         NaN        NaN  
6296         NaN        NaN  
7075         NaN        NaN

In [10]:
#中央値とかの説明
#https://marketer.jp/average-median.html
#中央値でうめる
#https://pyhoo.jp/dropna-fillna#%E6%AC%A0%E6%90%8D%E5%80%A4%E3%82%92%E5%88%97%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AE%E2%80%9D%E4%B8%AD%E5%A4%AE%E5%80%A4%E2%80%9D%E3%81%A7%E7%A9%B4%E5%9F%8B%E3%82%81
df_product_3 = df_product.copy()
df_product_3["unit_price"]=df_product_3["unit_price"].fillna(df_product_3["unit_price"].median()).round()
df_product_3["unit_cost"]=df_product_3["unit_cost"].fillna(df_product_3["unit_cost"].median()).round()
pd.merge(df_product_3, df_product_nan, on = "product_cd", suffixes=("_new2", "_NaN"))

product_cd  category_major_cd_new2  category_medium_cd_new2  \
0  P040802007                       4                      408   
1  P050103021                       5                      501   
2  P050405009                       5                      504   
3  P060802026                       6                      608   
4  P070202092                       7                      702   
5  P080504027                       8                      805   
6  P090204185                       9                      902   

   category_small_cd_new2  unit_price_new2  unit_cost_new2  \
0                   40802            252.0           189.0   
1                   50103            252.0           189.0   
2                   50405            252.0           189.0   
3                   60802            252.0           189.0   
4                   70202            252.0           189.0   
5                   80504            252.0           189.0   
6                   90204            252.0           189.0   

   category_major_cd_NaN  category_medium_cd_NaN  category_small_cd_NaN  \
0                      4                     408                  40802   
1                      5                     501                  50103   
2                      5                     504                  50405   
3                      6                     608                  60802   
4                      7                     702                  70202   
5                      8                     805                  80504   
6                      9                     902                  90204   

   unit_price_NaN  unit_cost_NaN  
0             NaN            NaN  
1             NaN            NaN  
2             NaN            NaN  
3             NaN            NaN  
4             NaN            NaN  
5             NaN            NaN  
6             NaN            NaN

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品の小区分（category_small_cd）ごとに算出した中央値で補完した新たなdf_product_4を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [24]:
df83_product = df_product.copy()



In [25]:
#https://deepage.net/features/pandas-reset-index.html
df83_1 = df83_product.groupby("category_small_cd").agg("median").round().reset_index()
df83_1

category_small_cd  category_major_cd  category_medium_cd  unit_price  \
0                40101                  4                 401       283.0   
1                40102                  4                 401       378.0   
2                40201                  4                 402       223.0   
3                40202                  4                 402       178.0   
4                40203                  4                 402       308.0   
..                 ...                ...                 ...         ...   
223              91401                  9                 914       315.0   
224              91402                  9                 914       900.0   
225              91403                  9                 914       652.0   
226              91501                  9                 915       510.0   
227              91503                  9                 915      1080.0   

     unit_cost  
0        212.0  
1        284.0  
2        168.0  
3        134.0  
4        231.0  
..         ...  
223      236.0  
224      675.0  
225      490.0  
226      383.0  
227      810.0  

[228 rows x 5 columns]

In [47]:
df_product_nan

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
159   P040802007                  4                 408              40802   
196   P050103021                  5                 501              50103   
496   P050405009                  5                 504              50405   
1531  P060802026                  6                 608              60802   
2012  P070202092                  7                 702              70202   
6296  P080504027                  8                 805              80504   
7075  P090204185                  9                 902              90204   

      unit_price  unit_cost  
159          NaN        NaN  
196          NaN        NaN  
496          NaN        NaN  
1531         NaN        NaN  
2012         NaN        NaN  
6296         NaN        NaN  
7075         NaN        NaN

In [48]:
df83_2 = pd.merge(df_product_nan[["product_cd","category_major_cd","category_medium_cd","category_small_cd"]],df83_1[["category_small_cd","unit_price","unit_cost"]],on = "category_small_cd")
df83_2

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040802007                  4                 408              40802   
1  P050103021                  5                 501              50103   
2  P050405009                  5                 504              50405   
3  P060802026                  6                 608              60802   
4  P070202092                  7                 702              70202   
5  P080504027                  8                 805              80504   
6  P090204185                  9                 902              90204   

   unit_price  unit_cost  
0       313.0      235.0  
1       132.0      100.0  
2       178.0      134.0  
3       270.0      200.0  
4       238.0      179.0  
5       258.0      196.0  
6       694.0      521.0

In [63]:
#https://ai-inter1.com/pandas-dataframe_union/
#df_product_1(NaN削除したもの)とdf83_2くっつける
df_product_4 = df_product_1.append(df83_2, ignore_index=True)
df_product_4

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0      P040101001                  4                 401              40101   
1      P040101002                  4                 401              40101   
2      P040101003                  4                 401              40101   
3      P040101004                  4                 401              40101   
4      P040101005                  4                 401              40101   
...           ...                ...                 ...                ...   
10025  P050405009                  5                 504              50405   
10026  P060802026                  6                 608              60802   
10027  P070202092                  7                 702              70202   
10028  P080504027                  8                 805              80504   
10029  P090204185                  9                 902              90204   

       unit_price  unit_cost  
0           198.0      149.0  
1           218.0      164.0  
2           230.0      173.0  
3           248.0      186.0  
4           268.0      201.0  
...           ...        ...  
10025       178.0      134.0  
10026       270.0      200.0  
10027       238.0      179.0  
10028       258.0      196.0  
10029       694.0      521.0  

[10030 rows x 6 columns]

In [73]:
df_product_4[df_product_4["product_cd"].isin(["P040802007","P050103021","P060802026"])]

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
10023  P040802007                  4                 408              40802   
10024  P050103021                  5                 501              50103   
10026  P060802026                  6                 608              60802   

       unit_price  unit_cost  
10023       313.0      235.0  
10024       132.0      100.0  
10026       270.0      200.0

---
> P-084: 顧客データフレーム（df_customer）の全顧客に対し、全期間の売上金額に占める2019年売上金額の割合を計算せよ。ただし、販売実績のない場合は0として扱うこと。そして計算した割合が0超のものを抽出せよ。 結果は10件表示させれば良い。また、作成したデータにNAやNANが存在しないことを確認せよ。

In [74]:
df84_receipt = df_receipt.copy()
df84_receipt.dtypes


sales_ymd          int64
sales_epoch        int64
store_cd          object
receipt_no         int64
receipt_sub_no     int64
customer_id       object
product_cd        object
quantity           int64
amount             int64
dtype: object

In [75]:
df84_receipt_2019 = df84_receipt[(df84_receipt["sales_ymd"] > 20181231) & (df84_receipt["sales_ymd"] < 20200101)]
df84_receipt_2019

sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
3        20190205   1549324800   S14042        1132               1   
5        20190605   1559692800   S13003        1112               1   
7        20190922   1569110400   S14040        1102               1   
9        20191010   1570665600   S14027        1102               1   
10       20190918   1568764800   S14025        1182               2   
...           ...          ...      ...         ...             ...   
104669   20190726   1564099200   S14006        1122               1   
104674   20190911   1568160000   S14046        1182               1   
104675   20190430   1556582400   S13041        1122               1   
104677   20190911   1568160000   S14047        1132               2   
104680   20190423   1555977600   S13016        1102               2   

           customer_id  product_cd  quantity  amount  
3       ZZ000000000000  P050301001         1      25  
5       CS003515000195  P050102002         1     138  
7       CS040415000178  P070501004         1     128  
9       CS027514000015  P071101003         1     680  
10      CS025415000134  P070401002         1     138  
...                ...         ...       ...     ...  
104669  CS006615000114  P071301002         1    2485  
104674  CS046415000017  P070703003         1      98  
104675  ZZ000000000000  P070503012         1     100  
104677  ZZ000000000000  P071006005         1     218  
104680  ZZ000000000000  P050601001         1     138  

[33835 rows x 9 columns]

In [78]:
df84_1 = pd.merge(df84_receipt_2019,df_customer["customer_id"],on = "customer_id")
df84_1.fillna(0)

sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
0       20190605   1559692800   S13003        1112               1   
1       20190605   1559692800   S13003        1112               2   
2       20190922   1569110400   S14040        1102               1   
3       20191001   1569888000   S14040        1192               1   
4       20190922   1569110400   S14040        1102               2   
...          ...          ...      ...         ...             ...   
19917   20190117   1547683200   S12013        1152               1   
19918   20190531   1559260800   S14010        1172               2   
19919   20190531   1559260800   S14010        1172               1   
19920   20190226   1551139200   S14034        1112               2   
19921   20190226   1551139200   S14034        1112               1   

          customer_id  product_cd  quantity  amount  
0      CS003515000195  P050102002         1     138  
1      CS003515000195  P070702001         1      81  
2      CS040415000178  P070501004         1     128  
3      CS040415000178  P040501002         1     180  
4      CS040415000178  P091401002         1      80  
...               ...         ...       ...     ...  
19917  CS013514000013  P071401001         1    1100  
19918  CS010615000104  P060802001         1     270  
19919  CS010615000104  P050104001         1     115  
19920  CS034605000004  P070705013         1     188  
19921  CS034605000004  P071401003         1    1100  

[19922 rows x 9 columns]

In [80]:
df84_2 = df84_1.groupby("customer_id").sum("amount").reset_index()
df84_2

customer_id  sales_ymd  sales_epoch  receipt_no  receipt_sub_no  \
0     CS001113000004   40380616   3104006400         224               3   
1     CS001114000005   40381462   3129062400         224               3   
2     CS001115000010   40380810   3108844800         224               3   
3     CS001205000004   80761874   6227539200        4728               6   
4     CS001205000006   40380448   3101932800        2364               3   
...              ...        ...          ...         ...             ...   
5014  CS051212000001   40382036   3142713600         224               3   
5015  CS051513000004   40381438   3126988800        2224               3   
5016  CS051515000002   40382050   3143923200        2304               3   
5017  CS052212000002   40382034   3142540800         224               3   
5018  CS052514000001   40381644   3132864000        2384               3   

      quantity  amount  
0            2    1298  
1            2     188  
2            2     578  
3            4     702  
4            2     486  
...        ...     ...  
5014         2     336  
5015         2     551  
5016         2     265  
5017         2     192  
5018         2     178  

[5019 rows x 7 columns]

In [85]:
df84_2019_amount = df84_receipt_2019["amount"].sum()
df84_2019_amount
df84_2["wariai"] = df84_2["amount"] / df84_2019_amount
df84_2[df84_2["wariai"] > 0].head(10)

customer_id  sales_ymd  sales_epoch  receipt_no  receipt_sub_no  \
0  CS001113000004   40380616   3104006400         224               3   
1  CS001114000005   40381462   3129062400         224               3   
2  CS001115000010   40380810   3108844800         224               3   
3  CS001205000004   80761874   6227539200        4728               6   
4  CS001205000006   40380448   3101932800        2364               3   
5  CS001211000025   40380644   3106425600         224               3   
6  CS001212000070   40382036   3142713600         224               3   
7  CS001214000009   80763224   6260198400         448               6   
8  CS001214000017  121142456   9329299200         672               9   
9  CS001214000048  121144082   9373190400         672               9   

   quantity  amount    wariai  
0         2    1298  0.000119  
1         2     188  0.000017  
2         2     578  0.000053  
3         4     702  0.000065  
4         2     486  0.000045  
5         2     456  0.000042  
6         2     456  0.000042  
7         4     664  0.000061  
8         6    2962  0.000273  
9         6    1889  0.000174

---
> P-085: 顧客データフレーム（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いて経度緯度変換用データフレーム（df_geocode）を紐付け、新たなdf_customer_1を作成せよ。ただし、複数紐づく場合は経度（longitude）、緯度（latitude）それぞれ平均を算出すること。


In [3]:
df85_geo = df_geocode.copy()
df85_geo

postal_cd prefecture      city  town street address     full_address  \
0       060-0000        北海道    札幌市中央区   NaN    NaN     NaN        北海道札幌市中央区   
1       064-0941        北海道    札幌市中央区   旭ケ丘    NaN     NaN     北海道札幌市中央区旭ケ丘   
2       060-0042        北海道    札幌市中央区   大通西    NaN     １丁目  北海道札幌市中央区大通西１丁目   
3       060-0042        北海道    札幌市中央区   大通西    NaN     ２丁目  北海道札幌市中央区大通西２丁目   
4       060-0042        北海道    札幌市中央区   大通西    NaN     ３丁目  北海道札幌市中央区大通西３丁目   
...          ...        ...       ...   ...    ...     ...              ...   
127247  907-1433        沖縄県   八重山郡竹富町  南風見仲    NaN     NaN   沖縄県八重山郡竹富町南風見仲   
127248  907-1751        沖縄県   八重山郡竹富町   波照間    NaN     NaN    沖縄県八重山郡竹富町波照間   
127249  907-1544        沖縄県   八重山郡竹富町    鳩間    NaN     NaN     沖縄県八重山郡竹富町鳩間   
127250  907-1800        沖縄県  八重山郡与那国町   NaN    NaN     NaN      沖縄県八重山郡与那国町   
127251  907-1801        沖縄県  八重山郡与那国町   与那国    NaN     NaN   沖縄県八重山郡与那国町与那国   

        longitude  latitude  
0       141.34103  43.05513  
1       141.31972  43.04223  
2       141.35637  43.06102  
3       141.35445  43.06080  
4       141.35275  43.06086  
...           ...       ...  
127247  123.88795  24.28820  
127248  123.78243  24.05859  
127249  123.82043  24.47225  
127250  123.00434  24.46812  
127251  122.98768  24.45592  

[127252 rows x 9 columns]

In [4]:
df85_cus = df_customer.copy()
df85_cus

customer_id customer_name  gender_cd gender   birth_day  age  \
0      CS021313000114        大野 あや子          1     女性  1981-04-29   37   
1      CS037613000071         六角 雅彦          9     不明  1952-04-01   66   
2      CS031415000172       宇多田 貴美子          1     女性  1976-10-04   42   
3      CS028811000001        堀井 かおり          1     女性  1933-03-27   86   
4      CS001215000145         田崎 美紀          1     女性  1995-03-29   24   
...               ...           ...        ...    ...         ...  ...   
21966  CS002512000474         市村 夏希          1     女性  1959-10-12   59   
21967  CS029414000065         上村 怜奈          1     女性  1970-10-19   48   
21968  CS012403000043          堀越 明          0     男性  1972-12-16   46   
21969  CS033512000184         池谷 華子          1     女性  1964-06-05   54   
21970  CS009213000022          稲垣 咲          1     女性  1996-08-16   22   

      postal_cd                  address application_store_cd  \
0      259-1113     神奈川県伊勢原市粟窪**********               S14021   
1      136-0076       東京都江東区南砂**********               S13037   
2      151-0053      東京都渋谷区代々木**********               S13031   
3      245-0016   神奈川県横浜市泉区和泉町**********               S14028   
4      144-0055      東京都大田区仲六郷**********               S13001   
...         ...                      ...                  ...   
21966  185-0034      東京都国分寺市光町**********               S13002   
21967  279-0043      千葉県浦安市富士見**********               S12029   
21968  231-0825  神奈川県横浜市中区本牧間門**********               S14012   
21969  245-0016   神奈川県横浜市泉区和泉町**********               S14033   
21970  154-0012      東京都世田谷区駒沢**********               S13009   

       application_date     status_cd  
0              20150905  0-00000000-0  
1              20150414  0-00000000-0  
2              20150529  D-20100325-C  
3              20160115  0-00000000-0  
4              20170605  6-20090929-2  
...                 ...           ...  
21966          20171110  0-00000000-0  
21967          20150313  F-20101028-F  
21968          20150406  0-00000000-0  
21969          20160206  0-00000000-0  
21970          20150424  0-00000000-0  

[21971 rows x 11 columns]

In [5]:
df85_geo_group = df85_geo.groupby("postal_cd").mean().reset_index()
df85_geo_group


postal_cd   longitude   latitude
0       001-0000  141.340380  43.090540
1       001-0010  141.350532  43.073257
2       001-0011  141.350248  43.074407
3       001-0012  141.349950  43.075545
4       001-0013  141.349655  43.076695
...          ...         ...        ...
120643  999-8522  139.889130  39.049920
120644  999-8523  139.924960  39.058220
120645  999-8524  139.902560  39.036790
120646  999-8525  139.909670  39.068510
120647  999-8531  139.873630  39.044780

[120648 rows x 3 columns]

In [6]:
df_customer_1 = pd.merge(df85_cus,df85_geo_group,on = "postal_cd")
df_customer_1

customer_id customer_name  gender_cd gender   birth_day  age  \
0      CS021313000114        大野 あや子          1     女性  1981-04-29   37   
1      CS021303000023          堀 一徳          0     男性  1980-04-05   38   
2      CS021303000007         石倉 俊二          0     男性  1987-07-04   31   
3      CS021313000183         草野 未來          1     女性  1986-12-21   32   
4      CS021314000098        筒井 れいな          1     女性  1985-09-21   33   
...               ...           ...        ...    ...         ...  ...   
21966  CS021612000057          星 朝陽          1     女性  1950-05-07   68   
21967  CS035401000016         白木 蒼甫          0     男性  1971-05-06   47   
21968  CS003612000043         竹内 那奈          1     女性  1952-07-12   66   
21969  CS007612000095         会田 陽子          9     不明  1950-12-08   68   
21970  CS021411000017         筒井 里奈          1     女性  1972-02-06   47   

      postal_cd                address application_store_cd  application_date  \
0      259-1113   神奈川県伊勢原市粟窪**********               S14021          20150905   
1      259-1113   神奈川県伊勢原市粟窪**********               S14021          20160411   
2      259-1113   神奈川県伊勢原市粟窪**********               S14021          20150707   
3      259-1113   神奈川県伊勢原市粟窪**********               S14021          20170611   
4      259-1113   神奈川県伊勢原市粟窪**********               S14021          20160901   
...         ...                    ...                  ...               ...   
21966  259-1134  神奈川県伊勢原市八幡台**********               S14021          20150405   
21967  155-0033    東京都世田谷区代田**********               S13035          20150527   
21968  182-0033   東京都調布市富士見町**********               S13003          20150608   
21969  276-0043    千葉県八千代市萱田**********               S12007          20150921   
21970  243-0121    神奈川県厚木市七沢**********               S14021          20150209   

          status_cd  longitude  latitude  
0      0-00000000-0  139.31779  35.41358  
1      0-00000000-0  139.31779  35.41358  
2      0-00000000-0  139.31779  35.41358  
3      0-00000000-0  139.31779  35.41358  
4      0-00000000-0  139.31779  35.41358  
...             ...        ...       ...  
21966  4-20100829-7  139.30992  35.38544  
21967  0-00000000-0  139.66281  35.65271  
21968  0-00000000-0  139.53539  35.66020  
21969  4-20100704-8  140.10959  35.73419  
21970  7-20090515-9  139.26601  35.44896  

[21971 rows x 13 columns]

---
> P-086: 前設問で作成した緯度経度つき顧客データフレーム（df_customer_1）に対し、申込み店舗コード（application_store_cd）をキーに店舗データフレーム（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客の緯度・経度を用いて距離（km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示すれば良い。

$$
緯度（ラジアン）：\phi \\
経度（ラジアン）：\lambda \\
距離L = 6371 * arccos(sin \phi_1 * sin \phi_2
+ cos \phi_1 * cos \phi_2 * cos(\lambda_1 − \lambda_2))
$$

In [7]:
df86 = pd.merge(df_customer_1, df_store, left_on="application_store_cd",right_on="store_cd", how="left")
df86_ans = df86[["customer_id", "address_x","longitude_x", "latitude_x", "address_y","longitude_y", "latitude_y"]]
# https://note.nkmk.me/python-numpy-sin-con-tan/
latitude_x = np.radians(df86_ans["latitude_x"])
latitude_y = np.radians(df86_ans["latitude_y"])
longitude_x = np.radians(df86_ans["longitude_x"])
longitude_y = np.radians(df86_ans["longitude_y"])
# https://note.nkmk.me/python-pandas-setting-with-copy-warning/
df86_ans["distance"] = 6371*np.arccos(np.sin(latitude_x) * np.sin(latitude_y) 
                                      + np.cos(latitude_x)* np.cos(latitude_y) * np.cos(longitude_x - longitude_y))

In [8]:
df86_ans

customer_id              address_x  longitude_x  latitude_x  \
0      CS021313000114   神奈川県伊勢原市粟窪**********    139.31779    35.41358   
1      CS021303000023   神奈川県伊勢原市粟窪**********    139.31779    35.41358   
2      CS021303000007   神奈川県伊勢原市粟窪**********    139.31779    35.41358   
3      CS021313000183   神奈川県伊勢原市粟窪**********    139.31779    35.41358   
4      CS021314000098   神奈川県伊勢原市粟窪**********    139.31779    35.41358   
...               ...                    ...          ...         ...   
21966  CS021612000057  神奈川県伊勢原市八幡台**********    139.30992    35.38544   
21967  CS035401000016    東京都世田谷区代田**********    139.66281    35.65271   
21968  CS003612000043   東京都調布市富士見町**********    139.53539    35.66020   
21969  CS007612000095    千葉県八千代市萱田**********    140.10959    35.73419   
21970  CS021411000017    神奈川県厚木市七沢**********    139.26601    35.44896   

            address_y  longitude_y  latitude_y  distance  
0      神奈川県伊勢原市伊勢原四丁目     139.3129    35.40169  1.394409  
1      神奈川県伊勢原市伊勢原四丁目     139.3129    35.40169  1.394409  
2      神奈川県伊勢原市伊勢原四丁目     139.3129    35.40169  1.394409  
3      神奈川県伊勢原市伊勢原四丁目     139.3129    35.40169  1.394409  
4      神奈川県伊勢原市伊勢原四丁目     139.3129    35.40169  1.394409  
...               ...          ...         ...       ...  
21966  神奈川県伊勢原市伊勢原四丁目     139.3129    35.40169  1.826997  
21967    東京都世田谷区用賀四丁目     139.6318    35.63029  3.750675  
21968   東京都狛江市和泉本町四丁目     139.5668    35.64462  3.324984  
21969       千葉県佐倉市上志津     140.1452    35.71872  3.645826  
21970  神奈川県伊勢原市伊勢原四丁目     139.3129    35.40169  6.758608  

[21971 rows x 8 columns]

---
> P-087:  顧客データフレーム（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなし、1顧客1レコードとなるように名寄せした名寄顧客データフレーム（df_customer_u）を作成せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残すものとし、売上金額合計が同一もしくは売上実績の無い顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [10]:
df87 = df_customer.copy()
df87_amount = df_receipt.groupby("customer_id")[["amount"]].sum().reset_index()
df87_amount.head()

customer_id  amount
0  CS001113000004    1298
1  CS001114000005     626
2  CS001115000010    3044
3  CS001205000004    1988
4  CS001205000006    3337

In [11]:
# https://note.nkmk.me/python-pandas-sort-values-sort-index/
df87_customer_amount = pd.merge(df87, df87_amount, on="customer_id",how="left").fillna(0).sort_values(["amount", "customer_id"],ascending=[False, True])
df87_customer_amount

customer_id customer_name  gender_cd gender   birth_day  age  \
16905  CS017415000097         福士 千夏          1     女性  1973-04-03   45   
12692  CS015415000185        岩淵 はるみ          1     女性  1973-09-19   45   
13550  CS031414000051        長澤 沙知絵          1     女性  1973-04-25   45   
4808   CS028415000007         紺野 あい          1     女性  1969-07-28   49   
14205  CS001605000009         安部 耕司          0     男性  1952-10-22   66   
...               ...           ...        ...    ...         ...  ...   
4377   CS053612000002        川島 ヒカル          1     女性  1957-12-18   61   
1537   CS053615000001         大森 郁恵          1     女性  1952-10-27   66   
3005   CS053615000003        田村 ひろ子          1     女性  1949-07-22   69   
7438   CS053702000002         松島 公顕          0     男性  1946-09-25   72   
9998   CS053714000001         戸塚 由宇          1     女性  1943-09-27   75   

      postal_cd                   address application_store_cd  \
16905  166-0014       東京都杉並区松ノ木**********               S13017   
12692  135-0043        東京都江東区塩浜**********               S13015   
13550  151-0064        東京都渋谷区上原**********               S13031   
4808   246-0023  神奈川県横浜市瀬谷区阿久和東**********               S14028   
14205  144-0035       東京都大田区南蒲田**********               S13001   
...         ...                       ...                  ...   
4377   272-0138       千葉県市川市南行徳**********               S12053   
1537   279-0023        千葉県浦安市高洲**********               S12053   
3005   279-0022        千葉県浦安市今川**********               S12053   
7438   279-0031        千葉県浦安市舞浜**********               S12053   
9998   279-0023        千葉県浦安市高洲**********               S12053   

       application_date     status_cd   amount  
16905          20151209  F-20101006-F  23086.0  
12692          20150322  F-20101014-F  20153.0  
13550          20150823  F-20101009-F  19202.0  
4808           20151212  F-20100922-F  19127.0  
14205          20160203  F-20101019-E  18925.0  
...                 ...           ...      ...  
4377           20180707  0-00000000-0      0.0  
1537           20180317  0-00000000-0      0.0  
3005           20190307  0-00000000-0      0.0  
7438           20180901  0-00000000-0      0.0  
9998           20180430  0-00000000-0      0.0  

[21971 rows x 12 columns]

In [13]:
# https://note.nkmk.me/python-pandas-duplicated-drop-duplicates/
df_customer_u = df87_customer_amount.drop_duplicates(subset=["customer_name", "postal_cd"], keep="first")
df_customer_u.head()

customer_id customer_name  gender_cd gender   birth_day  age  \
16905  CS017415000097         福士 千夏          1     女性  1973-04-03   45   
12692  CS015415000185        岩淵 はるみ          1     女性  1973-09-19   45   
13550  CS031414000051        長澤 沙知絵          1     女性  1973-04-25   45   
4808   CS028415000007         紺野 あい          1     女性  1969-07-28   49   
14205  CS001605000009         安部 耕司          0     男性  1952-10-22   66   

      postal_cd                   address application_store_cd  \
16905  166-0014       東京都杉並区松ノ木**********               S13017   
12692  135-0043        東京都江東区塩浜**********               S13015   
13550  151-0064        東京都渋谷区上原**********               S13031   
4808   246-0023  神奈川県横浜市瀬谷区阿久和東**********               S14028   
14205  144-0035       東京都大田区南蒲田**********               S13001   

       application_date     status_cd   amount  
16905          20151209  F-20101006-F  23086.0  
12692          20150322  F-20101014-F  20153.0  
13550          20150823  F-20101009-F  19202.0  
4808           20151212  F-20100922-F  19127.0  
14205          20160203  F-20101019-E  18925.0

---
> P-088: 前設問で作成したデータを元に、顧客データフレームに統合名寄IDを付与したデータフレーム（df_customer_n）を作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定

In [22]:
df_customer_n = pd.merge(df_customer[["customer_name","postal_cd","customer_id"]], df_customer_u[["customer_name","postal_cd","customer_id"]], how="left", on=["customer_name", "postal_cd"])
df_customer_n.sort_values(["customer_name", "postal_cd"],ascending=[False, True])


customer_name postal_cd   customer_id_x   customer_id_y
53            黒谷 麻緒  157-0067  CS008415000145  CS008415000145
18319         黒谷 陽子  204-0004  CS018311000042  CS018311000042
9283          黒谷 陽子  242-0024  CS025315000027  CS025315000027
11887         黒谷 華子  201-0012  CS003215000025  CS003215000025
2307          黒谷 華子  246-0035  CS033515000088  CS033515000088
...             ...       ...             ...             ...
2602       おかやま まひる  185-0012  CS002612000263  CS002612000263
13262      おかやま まひる  285-0858  CS007713000059  CS007713000059
7520       おかやま ひかり  185-0011  CS002515000380  CS002515000380
1317        おかやま そら  144-0046  CS001515000060  CS001515000060
13852      おかやま あさみ  176-0002  CS019415000213  CS019415000213

[21971 rows x 4 columns]

In [25]:
df_customer_n[(df_customer_n["customer_id_x"] != df_customer_n["customer_id_y"])]


customer_name postal_cd   customer_id_x   customer_id_y
713            黒崎 彩  167-0053  CS039613000119  CS001413000478
2013         久野 みゆき  144-0052  CS016712000025  CS001515000422
2752          役所 美優  279-0011  CS051513000005  CS029313000181
3271            原 優  166-0003  CS018413000015  CS017414000126
3484          森口 莉央  158-0083  CS035312000011  CS023213000021
3939           青木 亮  249-0001  CS022602000034  CS004315000066
4900           伴 芽以  144-0051  CS004712000149  CS001515000561
5385         宇野 真悠子  285-0855  CS007715000041  CS007315000087
5668         末永 真奈美  210-0807  CS023513000031  CS001414000204
6652         早美 あや子  136-0073  CS037313000104  CS015812000002
6705          金城 沙耶  166-0001  CS005315000005  CS005215000095
8183           深谷 遥  246-0037  CS035403000027  CS033815000031
9135          水谷 明慶  151-0071  CS031413000023  CS014411000046
11024          梶原 優  249-0001  CS022314000036  CS022415000110
11234          山内 優  144-0045  CS024315000174  CS001314000251
11847         宮下 陽子  115-0053  CS003502000148  CS020515000002
12001        田村 小百合  166-0001  CS039812000008  CS017512000138
14680        榊原 あさみ  212-0058  CS001311000109  CS001311000045
15005        米倉 菜々美  158-0097  CS035313000059  CS035312000145
15782        梅田 あさみ  246-0013  CS033515000257  CS025515000084
17163         小杉 愛梨  210-0011  CS037513000205  CS023513000029
17597          柴田 涼  166-0001  CS039511000003  CS005415000003
18337         波多野 花  144-0055  CS032215000084  CS001414000041
18933         小市 礼子  185-0013  CS002815000025  CS028414000005
19420        宇多田 文世  165-0027  CS023403000036  CS004315000058
20428         前田 美紀  185-0022  CS002615000172  CS002215000052
20498         野沢 莉沙  206-0812  CS003511000168  CS003411000030
21093         椎名 朝香  174-0063  CS017413000132  CS019412000157
21682        多部 あさみ  222-0032  CS010413000041  CS001412000304
21698          今 充則  246-0001  CS040601000007  CS038214000037

---
> P-閑話: df_customer_1, df_customer_nは使わないので削除する。

In [26]:
df_customer_1

customer_id customer_name  gender_cd gender   birth_day  age  \
0      CS021313000114        大野 あや子          1     女性  1981-04-29   37   
1      CS021303000023          堀 一徳          0     男性  1980-04-05   38   
2      CS021303000007         石倉 俊二          0     男性  1987-07-04   31   
3      CS021313000183         草野 未來          1     女性  1986-12-21   32   
4      CS021314000098        筒井 れいな          1     女性  1985-09-21   33   
...               ...           ...        ...    ...         ...  ...   
21966  CS021612000057          星 朝陽          1     女性  1950-05-07   68   
21967  CS035401000016         白木 蒼甫          0     男性  1971-05-06   47   
21968  CS003612000043         竹内 那奈          1     女性  1952-07-12   66   
21969  CS007612000095         会田 陽子          9     不明  1950-12-08   68   
21970  CS021411000017         筒井 里奈          1     女性  1972-02-06   47   

      postal_cd                address application_store_cd  application_date  \
0      259-1113   神奈川県伊勢原市粟窪**********               S14021          20150905   
1      259-1113   神奈川県伊勢原市粟窪**********               S14021          20160411   
2      259-1113   神奈川県伊勢原市粟窪**********               S14021          20150707   
3      259-1113   神奈川県伊勢原市粟窪**********               S14021          20170611   
4      259-1113   神奈川県伊勢原市粟窪**********               S14021          20160901   
...         ...                    ...                  ...               ...   
21966  259-1134  神奈川県伊勢原市八幡台**********               S14021          20150405   
21967  155-0033    東京都世田谷区代田**********               S13035          20150527   
21968  182-0033   東京都調布市富士見町**********               S13003          20150608   
21969  276-0043    千葉県八千代市萱田**********               S12007          20150921   
21970  243-0121    神奈川県厚木市七沢**********               S14021          20150209   

          status_cd  longitude  latitude  
0      0-00000000-0  139.31779  35.41358  
1      0-00000000-0  139.31779  35.41358  
2      0-00000000-0  139.31779  35.41358  
3      0-00000000-0  139.31779  35.41358  
4      0-00000000-0  139.31779  35.41358  
...             ...        ...       ...  
21966  4-20100829-7  139.30992  35.38544  
21967  0-00000000-0  139.66281  35.65271  
21968  0-00000000-0  139.53539  35.66020  
21969  4-20100704-8  140.10959  35.73419  
21970  7-20090515-9  139.26601  35.44896  

[21971 rows x 13 columns]

In [27]:
del df_customer_1

---
> P-089: 売上実績のある顧客に対し、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [28]:
df_customer_1

NameError: name 'df_customer_1' is not defined

In [29]:
del df_customer_n

---
> P-090: レシート明細データフレーム（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月のモデル構築用データを3セット作成せよ。

In [33]:
df90 = df_receipt.copy()
#https://note.nkmk.me/python-pandas-time-series-datetimeindex/
df90["sales_ymd_date"] = pd.to_datetime(df90["sales_ymd"],format="%Y%m%d")
df90




sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
0        20181103   1541203200   S14006         112               1   
1        20181118   1542499200   S13008        1132               2   
2        20170712   1499817600   S14028        1102               1   
3        20190205   1549324800   S14042        1132               1   
4        20180821   1534809600   S14025        1102               2   
...           ...          ...      ...         ...             ...   
104676   20180221   1519171200   S13043        1132               2   
104677   20190911   1568160000   S14047        1132               2   
104678   20170311   1489190400   S14040        1122               1   
104679   20170331   1490918400   S13002        1142               1   
104680   20190423   1555977600   S13016        1102               2   

           customer_id  product_cd  quantity  amount sales_ymd_date  
0       CS006214000001  P070305012         1     158     2018-11-03  
1       CS008415000097  P070701017         1      81     2018-11-18  
2       CS028414000014  P060101005         1     170     2017-07-12  
3       ZZ000000000000  P050301001         1      25     2019-02-05  
4       CS025415000050  P060102007         1      90     2018-08-21  
...                ...         ...       ...     ...            ...  
104676  ZZ000000000000  P050101001         1      40     2018-02-21  
104677  ZZ000000000000  P071006005         1     218     2019-09-11  
104678  CS040513000195  P050405003         1     168     2017-03-11  
104679  CS002513000049  P060303001         1     148     2017-03-31  
104680  ZZ000000000000  P050601001         1     138     2019-04-23  

[104681 rows x 10 columns]

In [36]:
type(df90.index)


pandas.core.indexes.range.RangeIndex

In [37]:
#https://note.nkmk.me/python-pandas-time-series-datetimeindex/
#https://note.nkmk.me/python-pandas-set-index/
df90.set_index("sales_ymd_date", inplace=True)

In [38]:
type(df90.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [53]:
#https://note.nkmk.me/python-pandas-time-series-multiindex/
df90_all = df90[["amount"]].resample('M').sum()
df90_all = df90_all.reset_index()
df90_all.dtypes



sales_ymd_date    datetime64[ns]
amount                     int64
dtype: object

In [55]:
df90_all["sales_ym"] = df90_all["sales_ymd_date"].dt.strftime('%Y%m')
df90_all

sales_ymd_date   amount sales_ym
0      2017-01-31   902056   201701
1      2017-02-28   764413   201702
2      2017-03-31   962945   201703
3      2017-04-30   847566   201704
4      2017-05-31   884010   201705
5      2017-06-30   894242   201706
6      2017-07-31   959205   201707
7      2017-08-31   954836   201708
8      2017-09-30   902037   201709
9      2017-10-31   905739   201710
10     2017-11-30   932157   201711
11     2017-12-31   939654   201712
12     2018-01-31   944509   201801
13     2018-02-28   864128   201802
14     2018-03-31   946588   201803
15     2018-04-30   937099   201804
16     2018-05-31  1004438   201805
17     2018-06-30  1012329   201806
18     2018-07-31  1058472   201807
19     2018-08-31  1045793   201808
20     2018-09-30   977114   201809
21     2018-10-31  1069939   201810
22     2018-11-30   967479   201811
23     2018-12-31  1016425   201812
24     2019-01-31  1064085   201901
25     2019-02-28   959538   201902
26     2019-03-31  1093753   201903
27     2019-04-30  1044210   201904
28     2019-05-31  1111985   201905
29     2019-06-30  1089063   201906
30     2019-07-31  1118371   201907
31     2019-08-31  1133614   201908
32     2019-09-30  1105696   201909
33     2019-10-31  1143062   201910

In [68]:
df90_study1 = df90_all[["sales_ym","amount"]][0:12]
df90_study2 = df90_all[["sales_ym","amount"]][12:24]
df90_study3 = df90_all[["sales_ym","amount"]][24:]





In [71]:
df90_study3

sales_ym   amount
24   201901  1064085
25   201902   959538
26   201903  1093753
27   201904  1044210
28   201905  1111985
29   201906  1089063
30   201907  1118371
31   201908  1133614
32   201909  1105696
33   201910  1143062

In [72]:
df90_test1 = df90_all[["sales_ym","amount"]][0:6]
df90_test2 = df90_all[["sales_ym","amount"]][6:12]
df90_test3 = df90_all[["sales_ym","amount"]][12:6]


In [73]:
df90_test1

sales_ym  amount
0   201701  902056
1   201702  764413
2   201703  962945
3   201704  847566
4   201705  884010
5   201706  894242

---
> P-091: 顧客データフレーム（df_customer）の各顧客に対し、売上実績のある顧客数と売上実績のない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [82]:
#Python基礎：アンダーサンプリング
#https://happy-analysis.com/python/python-topic-undersampling.html
#http://tekenuko.hatenablog.com/entry/2017/12/11/214522


df91_cus = df_customer.copy()
#amountの集計(顧客ごと)
df91_amount = df_receipt.groupby("customer_id")[["amount"]].sum().reset_index()
df91_amount.head()

#amountの集計したものと顧客情報くっつける
df91_tmp = pd.merge(df91_cus, df91_amount, how="left", on="customer_id")
df91_tmp
#
df91_tmp["buy_flg"] = df91_tmp["amount"].apply(lambda x: 0 if np.isnan(x) else 1)
print("0の件数:",len(df91_tmp[(df91_tmp["buy_flg"] == 0)]))
print("1の件数:",len(df91_tmp[(df91_tmp["buy_flg"] == 1)]))


0の件数: 13665
1の件数: 8306


In [83]:
# ライブラリのインポート
from imblearn.under_sampling import RandomUnderSampler

# サンプリング
rs=RandomUnderSampler(random_state=42)
df_sample,_=rs.fit_sample(df91_tmp,df91_tmp.buy_flg)
df_sample

customer_id customer_name  gender_cd gender   birth_day  age  \
0      CS005615000249         平田 華子          1     女性  1949-01-30   70   
1      CS019312000099        野田 めぐみ          1     女性  1983-01-04   36   
2      CS002715000196        喜多 かおり          1     女性  1944-08-09   74   
3      CS002502000037         窪田 広司          0     男性  1965-01-12   54   
4      CS002603000072         森永 光博          0     男性  1951-10-01   67   
...               ...           ...        ...    ...         ...  ...   
16607  CS002513000310         柳田 美嘉          1     女性  1963-09-17   55   
16608  CS005415000024          若林 花          1     女性  1969-03-03   50   
16609  CS001413000156        上原 恵望子          1     女性  1974-12-10   44   
16610  CS040314000065         谷川 紗季          1     女性  1987-01-14   32   
16611  CS029414000065         上村 怜奈          1     女性  1970-10-19   48   

      postal_cd                 address application_store_cd  \
0      167-0021      東京都杉並区井草**********               S13005   
1      173-0025     東京都板橋区熊野町**********               S13019   
2      184-0014   東京都小金井市貫井南町**********               S13002   
3      187-0004     東京都小平市天神町**********               S13002   
4      185-0002    東京都国分寺市東戸倉**********               S13002   
...         ...                     ...                  ...   
16607  187-0022    東京都小平市上水本町**********               S13002   
16608  167-0022     東京都杉並区下井草**********               S13005   
16609  144-0056     東京都大田区西六郷**********               S13001   
16610  226-0027  神奈川県横浜市緑区長津田**********               S14040   
16611  279-0043     千葉県浦安市富士見**********               S12029   

       application_date     status_cd  amount  buy_flg  
0              20170613  0-00000000-0     NaN        0  
1              20151009  0-00000000-0     NaN        0  
2              20170414  0-00000000-0     NaN        0  
3              20150626  0-00000000-0     NaN        0  
4              20160908  0-00000000-0     NaN        0  
...                 ...           ...     ...      ...  
16607          20160911  1-20100711-2   202.0        1  
16608          20150518  8-20100723-C  1598.0        1  
16609          20150724  1-20080210-1   203.0        1  
16610          20150703  1-20090421-4   243.0        1  
16611          20150313  F-20101028-F  7935.0        1  

[16612 rows x 13 columns]

In [84]:
print("0の件数:",len(df_sample[(df_sample["buy_flg"] == 0)]))
print("1の件数:",len(df_sample[(df_sample["buy_flg"] == 1)]))


0の件数: 8306
1の件数: 8306


---
> P-092: 顧客データフレーム（df_customer）では、性別に関する情報が非正規化の状態で保持されている。これを第三正規化せよ。

In [87]:

df92_cus = df_customer.copy()
df92_cus.head(10)


customer_id customer_name  gender_cd gender   birth_day  age postal_cd  \
0  CS021313000114        大野 あや子          1     女性  1981-04-29   37  259-1113   
1  CS037613000071         六角 雅彦          9     不明  1952-04-01   66  136-0076   
2  CS031415000172       宇多田 貴美子          1     女性  1976-10-04   42  151-0053   
3  CS028811000001        堀井 かおり          1     女性  1933-03-27   86  245-0016   
4  CS001215000145         田崎 美紀          1     女性  1995-03-29   24  144-0055   
5  CS020401000016         宮下 達士          0     男性  1974-09-15   44  174-0065   
6  CS015414000103         奥野 陽子          1     女性  1977-08-09   41  136-0073   
7  CS029403000008          釈 人志          0     男性  1973-08-17   45  279-0003   
8  CS015804000004         松谷 米蔵          0     男性  1931-05-02   87  136-0073   
9  CS033513000180          安斎 遥          1     女性  1962-07-11   56  241-0823   

                  address application_store_cd  application_date     status_cd  
0    神奈川県伊勢原市粟窪**********               S14021          20150905  0-00000000-0  
1      東京都江東区南砂**********               S13037          20150414  0-00000000-0  
2     東京都渋谷区代々木**********               S13031          20150529  D-20100325-C  
3  神奈川県横浜市泉区和泉町**********               S14028          20160115  0-00000000-0  
4     東京都大田区仲六郷**********               S13001          20170605  6-20090929-2  
5      東京都板橋区若木**********               S13020          20150225  0-00000000-0  
6      東京都江東区北砂**********               S13015          20150722  B-20100609-B  
7      千葉県浦安市海楽**********               S12029          20150515  0-00000000-0  
8      東京都江東区北砂**********               S13015          20150607  0-00000000-0  
9  神奈川県横浜市旭区善部町**********               S14033          20150728  6-20080506-5

In [88]:
df92_cus_2 = df92_cus.drop(columns="gender")
df92_cus_2

customer_id customer_name  gender_cd   birth_day  age postal_cd  \
0      CS021313000114        大野 あや子          1  1981-04-29   37  259-1113   
1      CS037613000071         六角 雅彦          9  1952-04-01   66  136-0076   
2      CS031415000172       宇多田 貴美子          1  1976-10-04   42  151-0053   
3      CS028811000001        堀井 かおり          1  1933-03-27   86  245-0016   
4      CS001215000145         田崎 美紀          1  1995-03-29   24  144-0055   
...               ...           ...        ...         ...  ...       ...   
21966  CS002512000474         市村 夏希          1  1959-10-12   59  185-0034   
21967  CS029414000065         上村 怜奈          1  1970-10-19   48  279-0043   
21968  CS012403000043          堀越 明          0  1972-12-16   46  231-0825   
21969  CS033512000184         池谷 華子          1  1964-06-05   54  245-0016   
21970  CS009213000022          稲垣 咲          1  1996-08-16   22  154-0012   

                       address application_store_cd  application_date  \
0         神奈川県伊勢原市粟窪**********               S14021          20150905   
1           東京都江東区南砂**********               S13037          20150414   
2          東京都渋谷区代々木**********               S13031          20150529   
3       神奈川県横浜市泉区和泉町**********               S14028          20160115   
4          東京都大田区仲六郷**********               S13001          20170605   
...                        ...                  ...               ...   
21966      東京都国分寺市光町**********               S13002          20171110   
21967      千葉県浦安市富士見**********               S12029          20150313   
21968  神奈川県横浜市中区本牧間門**********               S14012          20150406   
21969   神奈川県横浜市泉区和泉町**********               S14033          20160206   
21970      東京都世田谷区駒沢**********               S13009          20150424   

          status_cd  
0      0-00000000-0  
1      0-00000000-0  
2      D-20100325-C  
3      0-00000000-0  
4      6-20090929-2  
...             ...  
21966  0-00000000-0  
21967  F-20101028-F  
21968  0-00000000-0  
21969  0-00000000-0  
21970  0-00000000-0  

[21971 rows x 10 columns]

---
> P-093: 商品データフレーム（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータフレーム（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データフレームを作成せよ。

In [89]:
df93_p = df_product.copy()
df93_p.head(10)

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040101001                  4                 401              40101   
1  P040101002                  4                 401              40101   
2  P040101003                  4                 401              40101   
3  P040101004                  4                 401              40101   
4  P040101005                  4                 401              40101   
5  P040101006                  4                 401              40101   
6  P040101007                  4                 401              40101   
7  P040101008                  4                 401              40101   
8  P040101009                  4                 401              40101   
9  P040101010                  4                 401              40101   

   unit_price  unit_cost  
0       198.0      149.0  
1       218.0      164.0  
2       230.0      173.0  
3       248.0      186.0  
4       268.0      201.0  
5       298.0      224.0  
6       338.0      254.0  
7       420.0      315.0  
8       498.0      374.0  
9       580.0      435.0

In [90]:
df93_cat = df_category.copy()
df93_cat

category_major_cd category_major_name  category_medium_cd  \
0                    4                  惣菜                 401   
1                    4                  惣菜                 401   
2                    4                  惣菜                 402   
3                    4                  惣菜                 402   
4                    4                  惣菜                 402   
..                 ...                 ...                 ...   
223                  9                  洗剤                 914   
224                  9                  洗剤                 914   
225                  9                  洗剤                 914   
226                  9                  洗剤                 915   
227                  9                  洗剤                 915   

    category_medium_name  category_small_cd category_small_name  
0                    御飯類              40101                 弁当類  
1                    御飯類              40102                 寿司類  
2                    佃煮類              40201               魚介佃煮類  
3                    佃煮類              40202               海草佃煮類  
4                    佃煮類              40203               野菜佃煮類  
..                   ...                ...                 ...  
223               ペットフード              91401              ペットフード  
224               ペットフード              91402               ペット用剤  
225               ペットフード              91403               ペット用具  
226                ＤＩＹ用品              91501             建築・塗装材料  
227                ＤＩＹ用品              91503                園芸用品  

[228 rows x 6 columns]

In [92]:
#https://note.nkmk.me/python-pandas-merge-join/
df93_p2 = pd.merge(df93_p, df93_cat, how="left", on=["category_major_cd","category_medium_cd","category_small_cd"])
df93_p2

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0      P040101001                  4                 401              40101   
1      P040101002                  4                 401              40101   
2      P040101003                  4                 401              40101   
3      P040101004                  4                 401              40101   
4      P040101005                  4                 401              40101   
...           ...                ...                 ...                ...   
10025  P091503001                  9                 915              91503   
10026  P091503002                  9                 915              91503   
10027  P091503003                  9                 915              91503   
10028  P091503004                  9                 915              91503   
10029  P091503005                  9                 915              91503   

       unit_price  unit_cost category_major_name category_medium_name  \
0           198.0      149.0                  惣菜                  御飯類   
1           218.0      164.0                  惣菜                  御飯類   
2           230.0      173.0                  惣菜                  御飯類   
3           248.0      186.0                  惣菜                  御飯類   
4           268.0      201.0                  惣菜                  御飯類   
...           ...        ...                 ...                  ...   
10025       280.0      210.0                  洗剤                ＤＩＹ用品   
10026       680.0      510.0                  洗剤                ＤＩＹ用品   
10027      1080.0      810.0                  洗剤                ＤＩＹ用品   
10028      1130.0      848.0                  洗剤                ＤＩＹ用品   
10029      1280.0      960.0                  洗剤                ＤＩＹ用品   

      category_small_name  
0                     弁当類  
1                     弁当類  
2                     弁当類  
3                     弁当類  
4                     弁当類  
...                   ...  
10025                園芸用品  
10026                園芸用品  
10027                園芸用品  
10028                園芸用品  
10029                園芸用品  

[10030 rows x 9 columns]

---
> P-094: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [96]:
df93_p2.to_csv("./data/df94_UTF-8_header.csv", encoding="UTF-8", index=False)

---
> P-095: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはCP932

In [97]:
df93_p2.to_csv("./data/df95_CP932_header.csv", encoding="CP932", index=False)

---
> P-096: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [98]:
df93_p2.to_csv("./data/df96_UTF-8_data.csv", encoding="UTF-8", header=False, index=False)

---
> P-097: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [100]:
# "./data/df94_UTF-8_header.csv" の読み込み
df97 = pd.read_csv("./data/df94_UTF-8_header.csv" )
df97.head(10)

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040101001                  4                 401              40101   
1  P040101002                  4                 401              40101   
2  P040101003                  4                 401              40101   
3  P040101004                  4                 401              40101   
4  P040101005                  4                 401              40101   
5  P040101006                  4                 401              40101   
6  P040101007                  4                 401              40101   
7  P040101008                  4                 401              40101   
8  P040101009                  4                 401              40101   
9  P040101010                  4                 401              40101   

   unit_price  unit_cost category_major_name category_medium_name  \
0       198.0      149.0                  惣菜                  御飯類   
1       218.0      164.0                  惣菜                  御飯類   
2       230.0      173.0                  惣菜                  御飯類   
3       248.0      186.0                  惣菜                  御飯類   
4       268.0      201.0                  惣菜                  御飯類   
5       298.0      224.0                  惣菜                  御飯類   
6       338.0      254.0                  惣菜                  御飯類   
7       420.0      315.0                  惣菜                  御飯類   
8       498.0      374.0                  惣菜                  御飯類   
9       580.0      435.0                  惣菜                  御飯類   

  category_small_name  
0                 弁当類  
1                 弁当類  
2                 弁当類  
3                 弁当類  
4                 弁当類  
5                 弁当類  
6                 弁当類  
7                 弁当類  
8                 弁当類  
9                 弁当類

---
> P-098: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [105]:
# "./data/df96_UTF-8_data.csv" の読み込み
df98 = pd.read_csv("./data/df96_UTF-8_data.csv", header=None, 
                   names=("product_cd","category_major_cd","category_medium_cd",
                          "category_small_cd","unit_price","unit_cost","category_major_name","category_medium_name","category_small_name"))
df98.head(10)

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040101001                  4                 401              40101   
1  P040101002                  4                 401              40101   
2  P040101003                  4                 401              40101   
3  P040101004                  4                 401              40101   
4  P040101005                  4                 401              40101   
5  P040101006                  4                 401              40101   
6  P040101007                  4                 401              40101   
7  P040101008                  4                 401              40101   
8  P040101009                  4                 401              40101   
9  P040101010                  4                 401              40101   

   unit_price  unit_cost category_major_name category_medium_name  \
0       198.0      149.0                  惣菜                  御飯類   
1       218.0      164.0                  惣菜                  御飯類   
2       230.0      173.0                  惣菜                  御飯類   
3       248.0      186.0                  惣菜                  御飯類   
4       268.0      201.0                  惣菜                  御飯類   
5       298.0      224.0                  惣菜                  御飯類   
6       338.0      254.0                  惣菜                  御飯類   
7       420.0      315.0                  惣菜                  御飯類   
8       498.0      374.0                  惣菜                  御飯類   
9       580.0      435.0                  惣菜                  御飯類   

  category_small_name  
0                 弁当類  
1                 弁当類  
2                 弁当類  
3                 弁当類  
4                 弁当類  
5                 弁当類  
6                 弁当類  
7                 弁当類  
8                 弁当類  
9                 弁当類

---
> P-099: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [106]:
#https://note.nkmk.me/python-pandas-to-csv/
df93_p2.to_csv("./data/df99_UTF-8_data.csv", encoding="UTF-8", index=False,sep ="\t")

---
> P-100: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [109]:
#https://note.nkmk.me/python-pandas-read-csv-tsv/
df100 = pd.read_table("./data/df99_UTF-8_data.csv")
df100.head(10)

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040101001                  4                 401              40101   
1  P040101002                  4                 401              40101   
2  P040101003                  4                 401              40101   
3  P040101004                  4                 401              40101   
4  P040101005                  4                 401              40101   
5  P040101006                  4                 401              40101   
6  P040101007                  4                 401              40101   
7  P040101008                  4                 401              40101   
8  P040101009                  4                 401              40101   
9  P040101010                  4                 401              40101   

   unit_price  unit_cost category_major_name category_medium_name  \
0       198.0      149.0                  惣菜                  御飯類   
1       218.0      164.0                  惣菜                  御飯類   
2       230.0      173.0                  惣菜                  御飯類   
3       248.0      186.0                  惣菜                  御飯類   
4       268.0      201.0                  惣菜                  御飯類   
5       298.0      224.0                  惣菜                  御飯類   
6       338.0      254.0                  惣菜                  御飯類   
7       420.0      315.0                  惣菜                  御飯類   
8       498.0      374.0                  惣菜                  御飯類   
9       580.0      435.0                  惣菜                  御飯類   

  category_small_name  
0                 弁当類  
1                 弁当類  
2                 弁当類  
3                 弁当類  
4                 弁当類  
5                 弁当類  
6                 弁当類  
7                 弁当類  
8                 弁当類  
9                 弁当類

# これで１００本終わりです。おつかれさまでした！